In [ ]:
# dependency_bootstrap_marker
# ─── Dependency Bootstrap (auto-installs missing lightweight libs) ───
import importlib.util, subprocess, sys
required = ['requests', 'statsmodels', 'pyarrow', 'yfinance', 'seaborn', 'tqdm', 'pandas_datareader', 'fuzzywuzzy']
missing = [m for m in required if importlib.util.find_spec(m) is None]
if missing:
    print('Installing missing packages:', missing)
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + missing)
    except Exception as e:
        print('Auto-install failed — install manually if needed:', e)


# Congressional Trading Signal Strategy
## QPM Final Project — Chicago Booth

**Objective:** Build and validate a systematic ETF trading strategy that aggregates Congressional trading disclosures into sector-level signals, tests whether those signals contain exploitable alpha, and evaluates strategy performance against standard factor models.

**Hypotheses:**
- **H1 (Market Underreaction):** `CAR[-1,+1] ≈ 0` but `CAR[+2,+20] > 0` for purchases — the market underreacts to congressional trade disclosures
- **H2 (Committee Advantage):** Committee-relevant trades generate higher CARs than non-relevant trades

**Pipeline:** Foundation → H1 Event Study → H2 Committee Analysis → Signal Engine → Backtest → Factor Regression

**Critical Constraint:** All signals use `ReportDate` (disclosure date) — NEVER `TransactionDate`. The STOCK Act allows up to 45-day disclosure lag; this is a feature of the signal, not a bug.

In [ ]:
# ─── Imports ───
import os
import json
import warnings
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
try:
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
except ImportError:
    sm = None
    smf = None
    print('statsmodels not available; regression modules disabled.')
from scipy import stats
from datetime import datetime, timedelta
from pathlib import Path


try:
    import requests
except ImportError:
    import json as _json
    import urllib.request as _urlreq
    class _Resp:
        def __init__(self, resp):
            self._resp = resp
            self.status_code = getattr(resp, 'status', None)
            self.text = resp.read().decode('utf-8', errors='ignore')
        def json(self):
            return _json.loads(self.text)
        def raise_for_status(self):
            if self.status_code and self.status_code >= 400:
                raise Exception(f'Status {self.status_code}')
    class _RequestsShim:
        def get(self, url, headers=None, timeout=None):
            req = _urlreq.Request(url, headers=headers or {})
            with _urlreq.urlopen(req, timeout=timeout) as r:
                return _Resp(r)
    requests = _RequestsShim()
    print('requests not available; using urllib shim (limited).')

try:
    import yfinance as yf
except ImportError:
    yf = None
    print('yfinance not available; price downloads disabled.')

try:
    import seaborn as sns
except ImportError:
    sns = None
    print('seaborn not available; using matplotlib defaults.')

if sns is None:
    class _DummySeaborn:
        def heatmap(self, data, cmap=None, center=None, vmin=None, vmax=None, ax=None,
                   xticklabels=True, yticklabels=True, cbar_kws=None, linewidths=0,
                   annot=False, fmt='.2g'):
            import numpy as _np
            import matplotlib.pyplot as _plt
            ax = ax or _plt.gca()
            arr = data.to_numpy() if hasattr(data, 'to_numpy') else _np.array(data)
            im = ax.imshow(arr, cmap=cmap, vmin=vmin, vmax=vmax, aspect='auto')
            if cbar_kws is None:
                cbar_kws = {}
            ax.figure.colorbar(im, ax=ax, **cbar_kws)
            if xticklabels is not False:
                labels = data.columns if hasattr(data, 'columns') else range(arr.shape[1])
                ax.set_xticks(range(len(labels)))
                ax.set_xticklabels(labels)
            if yticklabels is not False:
                labels = data.index if hasattr(data, 'index') else range(arr.shape[0])
                ax.set_yticks(range(len(labels)))
                ax.set_yticklabels(labels)
            if annot:
                for r in range(arr.shape[0]):
                    for c in range(arr.shape[1]):
                        ax.text(c, r, format(arr[r, c], fmt), ha='center', va='center')
            return im
    sns = _DummySeaborn()

try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x, **k: x
    print('tqdm not available; progress bars disabled.')

try:
    import pandas_datareader.data as web
except ImportError:
    web = None

try:
    from fuzzywuzzy import fuzz, process as fuzz_process
except ImportError:
    fuzz = None
    fuzz_process = None

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)
plt.style.use('seaborn-v0_8-whitegrid')

# ─── API & Environment ───
def _load_env_file(path: Path = Path('.env')) -> None:
    """Load KEY=VALUE pairs from .env into process env if missing."""
    if not path.exists():
        return
    for raw in path.read_text().splitlines():
        line = raw.strip()
        if not line or line.startswith('#') or '=' not in line:
            continue
        key, value = line.split('=', 1)
        key = key.strip()
        value = value.strip().strip('\"').strip("'")
        os.environ.setdefault(key, value)

_load_env_file()
QUIVER_API_KEY = (os.environ.get('QUIVER_API_KEY') or '').strip()
CONGRESS_API_KEY = (os.environ.get('CONGRESS_API_KEY') or '').strip()
QUIVER_BASE_URL = 'https://api.quiverquant.com/beta'

# ─── Sample Period ───
SAMPLE_START          = '2016-01-01'
SAMPLE_END            = '2025-12-31'
ESTIMATION_WINDOW_START = '2015-01-01'

# ─── Event Study ───
ESTIMATION_WINDOW      = (-250, -30)
EVENT_WINDOW_IMMEDIATE = (-1, 1)
EVENT_WINDOW_DRIFT     = (2, 20)
MIN_ESTIMATION_DAYS    = 120

# ─── Signal Engine ───
LOOKBACK_DAYS           = 45
CONVICTION_THRESHOLD    = 0.80
BACKTEST_LOOKBACK_DAYS  = 90

# ─── Backtest ───
REBALANCE_FREQUENCY   = 'ME'
TRANSACTION_COST_BPS  = 10

# ─── Amount Midpoints — STOCK Act range mapping ───
# ASSUMPTION A1: Use midpoint of each STOCK Act range as dollar estimate.
# Raw Amount is a lower bound; midpoints reduce systematic downward bias.
AMOUNT_MIDPOINTS = {
    1001:     8000,
    15001:    32500,
    50001:    75000,
    100001:   175000,
    250001:   375000,
    500001:   750000,
    1000001:  3000000,
    5000001:  15000000,
    25000001: 37500000,
    50000001: 50000001,
}

def get_amount_midpoint(amount):
    """Map STOCK Act lower bound to midpoint estimate (ASSUMPTION A1)."""
    for lower in sorted(AMOUNT_MIDPOINTS.keys(), reverse=True):
        if amount >= lower:
            return AMOUNT_MIDPOINTS[lower]
    return amount

# ─── Sector Configuration ───
# ASSUMPTION A2: Restrict to 7 sectors with high government intervention.
# This is a design choice — document edge cases in assumptions.md.
SECTOR_CONFIG = {
    'Defense & Aerospace': {
        'target_etf': 'XAR',
        'committees': [
            'House Armed Services', 'Senate Armed Services',
            'House Appropriations', 'Senate Appropriations',
            'House Foreign Affairs', 'Senate Foreign Relations',
        ],
        'gics_sector': 'Industrials',
        'gics_industry_keywords': ['Aerospace', 'Defense'],
        'tau_member': 2, 'tau_ticker': 3, 'min_transactions': 8,
    },
    'Healthcare & Pharmaceuticals': {
        'target_etf': 'XHS',
        'committees': [
            'House Energy and Commerce', 'Senate HELP',
            'House Ways and Means', 'Senate Finance',
        ],
        'gics_sector': 'Health Care',
        'gics_industry_keywords': ['Health Care', 'Pharmaceuticals', 'Biotechnology', 'Medical'],
        'tau_member': 2, 'tau_ticker': 3, 'min_transactions': 8,
    },
    'Technology': {
        'target_etf': 'XLK',
        'committees': [
            'House Science, Space, and Technology', 'Senate Commerce',
            'House Judiciary', 'Senate Judiciary',
            'House Energy and Commerce',
        ],
        'gics_sector': 'Information Technology',
        'gics_industry_keywords': ['Software', 'Technology', 'Semiconductor', 'Hardware', 'IT Services'],
        'tau_member': 2, 'tau_ticker': 3, 'min_transactions': 8,
    },
    'Financial Services': {
        'target_etf': 'XLF',
        'committees': [
            'House Financial Services', 'Senate Banking',
            'House Ways and Means', 'Senate Finance',
        ],
        'gics_sector': 'Financials',
        'gics_industry_keywords': ['Banks', 'Insurance', 'Capital Markets', 'Financial'],
        'tau_member': 2, 'tau_ticker': 3, 'min_transactions': 8,
    },
    'Energy': {
        'target_etf': 'XLE',
        'committees': [
            'House Energy and Commerce', 'Senate Energy and Natural Resources',
            'House Natural Resources', 'Senate Environment and Public Works',
        ],
        'gics_sector': 'Energy',
        'gics_industry_keywords': ['Oil', 'Gas', 'Energy'],
        'tau_member': 2, 'tau_ticker': 3, 'min_transactions': 8,
    },
    'Utilities': {
        'target_etf': 'XLU',
        'committees': [
            'House Energy and Commerce', 'Senate Energy and Natural Resources',
            'House Natural Resources', 'Senate Environment and Public Works',
        ],
        'gics_sector': 'Utilities',
        'gics_industry_keywords': ['Electric', 'Gas', 'Water', 'Utilities'],
        'tau_member': 2, 'tau_ticker': 3, 'min_transactions': 5,
    },
    'Industrials': {
        'target_etf': 'XLI',
        'committees': [
            'House Transportation and Infrastructure',
            'Senate Commerce, Science, and Transportation',
            'House Armed Services', 'Senate Armed Services',
        ],
        'gics_sector': 'Industrials',
        'gics_industry_keywords': ['Industrial', 'Manufacturing', 'Transportation', 'Construction'],
        'tau_member': 2, 'tau_ticker': 3, 'min_transactions': 8,
    },
}

ETF_TO_SECTOR = {v['target_etf']: k for k, v in SECTOR_CONFIG.items()}

# ─── Paths ───
DATA_DIR      = Path('data')
RAW_DIR       = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
EXTERNAL_DIR  = DATA_DIR / 'external'
LOGS_DIR      = Path('logs')

for d in [RAW_DIR, PROCESSED_DIR, EXTERNAL_DIR, LOGS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print('Configuration loaded.')
print(f'Sample period: {SAMPLE_START} to {SAMPLE_END}')
print(f'Sectors tracked: {list(SECTOR_CONFIG.keys())}')

---
## Phase 1: Data Pipeline
### Modules: Quiver Client → Sector Mapper → Committee Mapper → Returns → Enrichment

**Definition of Done:**
- All data sources fetched, validated, and persisted to `data/processed/`
- Master enriched dataset joins trades → sectors → committees
- Schema validation passes with zero nulls in critical columns

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 1: Quiver Quant API Client
# ═══════════════════════════════════════════════════════════════════════════
#
# Verified API schema (live endpoint, 2026-02-22):
#   Columns: Representative, BioGuideID, ReportDate, TransactionDate, Ticker,
#            Transaction, Range, House, Amount, Party, last_modified, TickerType,
#            Description, ExcessReturn, PriceChange, SPYChange
#   Auth:    X-CSRFToken header + User-Agent required (Cloudflare blocks without UA)
#   Dates:   Already named ReportDate / TransactionDate — no rename needed
#   Chamber: 'House' column contains 'Representatives' or 'Senate'
#   Transactions: 'Purchase', 'Sale', 'Sale (Full)', 'Sale (Partial)', 'Exchange'
#
# NORMALIZATION applied immediately on fetch:
#   Representative → Name
#   House          → Chamber  ('Representatives' → 'House', 'Senate' stays)
#   Sale (Full) / Sale (Partial) → Sale
#   Exchange → dropped (not a directional trade)

QUIVER_HEADERS = {
    'accept':     'application/json',
        'User-Agent': 'Mozilla/5.0',   # Required — Cloudflare blocks without User-Agent
}


def fetch_live_trades(api_key: str) -> pd.DataFrame:
    """
    Fetch congressional trades from Quiver Quant /live/congresstrading endpoint.
    Returns normalized DataFrame (Name, Chamber, ReportDate, TransactionDate, ...).
    """
    headers = {**QUIVER_HEADERS, 'Authorization': f'Token {api_key}'}
    url = f'{QUIVER_BASE_URL}/live/congresstrading'
    try:
        resp = requests.get(url, headers=headers, timeout=30)
        resp.raise_for_status()
        df = pd.DataFrame(resp.json())
    except Exception as e:
        print(f'[fetch_live_trades] Request failed: {e}')
        return pd.DataFrame()

    return _normalize_columns(df)


def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize Quiver API column names to internal schema.
    Handles both current API schema and legacy parquet files.
    """
    df = df.copy()

    # Legacy: Filed/Traded → ReportDate/TransactionDate
    if 'Filed' in df.columns:
        df = df.rename(columns={'Filed': 'ReportDate', 'Traded': 'TransactionDate'})

    # Defensive: raise if date columns are still missing
    if 'ReportDate' not in df.columns:
        raise ValueError(f'Missing ReportDate. Found columns: {list(df.columns)}')

    # Representative → Name
    if 'Representative' in df.columns and 'Name' not in df.columns:
        df = df.rename(columns={'Representative': 'Name'})

    # House → Chamber; normalize values
    if 'House' in df.columns and 'Chamber' not in df.columns:
        df = df.rename(columns={'House': 'Chamber'})
    if 'Chamber' in df.columns:
        df['Chamber'] = df['Chamber'].replace({'Representatives': 'House'})

    # Normalize sale transaction types
    df['Transaction'] = df['Transaction'].replace({
        'Sale (Full)':    'Sale',
        'Sale (Partial)': 'Sale',
    })

    # Parse dates
    for col in ['ReportDate', 'TransactionDate']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    return df


def load_historical(parquet_path) -> pd.DataFrame:
    """Load historical trades from parquet; normalize column names."""
    path = Path(parquet_path)
    if not path.exists():
        return pd.DataFrame()
    df = pd.read_parquet(path)
    return _normalize_columns(df)


def sync_and_deduplicate(df_hist: pd.DataFrame, df_live: pd.DataFrame) -> pd.DataFrame:
    """
    Merge historical + live data, then deduplicate.

    Dedup strategy:
      1. Native transaction ID ('id', 'transaction_id') if present
      2. Composite key: (Name, Ticker, TransactionDate, Transaction, Amount)
         — includes Amount to preserve different-sized same-day trades (bug fix)
    """
    if df_hist.empty:
        df = df_live.copy()
    elif df_live.empty:
        df = df_hist.copy()
    else:
        df = pd.concat([df_hist, df_live], ignore_index=True)

    id_col = next((c for c in ['transaction_id', 'id'] if c in df.columns), None)
    if id_col:
        df = df.drop_duplicates(subset=[id_col])
    else:
        dedup_cols = [c for c in ['Name', 'Ticker', 'TransactionDate', 'Transaction', 'Amount']
                      if c in df.columns]
        df = df.drop_duplicates(subset=dedup_cols)

    return df.reset_index(drop=True)


def validate_and_enrich_raw(df: pd.DataFrame) -> pd.DataFrame:
    """Validate raw trades; add Direction, AmountMidpoint, DisclosureLag."""
    df = df.copy()

    # Keep only Purchase and Sale (drop Exchange and anything else)
    df = df[df['Transaction'].isin({'Purchase', 'Sale'})].copy()

    # Direction encoding
    df['Direction'] = df['Transaction'].map({'Purchase': 1, 'Sale': -1})

    # Amount midpoint (ASSUMPTION A1: midpoint of STOCK Act range)
    if 'Amount' in df.columns:
        df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce').fillna(0)
        df['AmountMidpoint'] = df['Amount'].apply(get_amount_midpoint)

    # Disclosure lag in calendar days
    if 'TransactionDate' in df.columns:
        df['DisclosureLag'] = (df['ReportDate'] - df['TransactionDate']).dt.days
    else:
        df['DisclosureLag'] = np.nan

    # Drop rows missing critical columns
    critical = [c for c in ['ReportDate', 'Ticker', 'Name', 'Transaction'] if c in df.columns]
    df = df.dropna(subset=critical)

    # Filter to sample period using ReportDate only (ASSUMPTION A5 — no lookahead)
    df = df[
        (df['ReportDate'] >= SAMPLE_START) &
        (df['ReportDate'] <= SAMPLE_END)
    ].copy()

    return df.reset_index(drop=True)


# ─── Load Data ───
print('Loading congressional trading data...')

df_hist = load_historical(RAW_DIR / 'congress_trading.parquet')

if QUIVER_API_KEY:
    print(f'Fetching live data from Quiver Quant API...')
    df_live = fetch_live_trades(QUIVER_API_KEY)
    if not df_live.empty:
        print(f'  Live records fetched: {len(df_live):,}')
    df_raw = sync_and_deduplicate(df_hist, df_live)
elif not df_hist.empty:
    df_raw = df_hist.copy()
else:
    raise FileNotFoundError(
        'No data source available. Set QUIVER_API_KEY or provide data/raw/congress_trading.parquet.'
    )

df_raw = validate_and_enrich_raw(df_raw)
print(f'\nRaw trades loaded and validated: {len(df_raw):,}')
print(f'Date range: {df_raw["ReportDate"].min().date()} to {df_raw["ReportDate"].max().date()}')
print(f'Purchases: {(df_raw["Direction"]==1).sum():,} | Sales: {(df_raw["Direction"]==-1).sum():,}')
df_raw.head()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 2: Sector Mapper — Ticker → GICS → Project Sector
# ═══════════════════════════════════════════════════════════════════════════

GICS_CACHE_PATH = RAW_DIR / 'yfinance_gics_cache.json'

def _load_gics_cache() -> dict:
    if GICS_CACHE_PATH.exists():
        with open(GICS_CACHE_PATH) as f:
            return json.load(f)
    return {}

def _save_gics_cache(cache: dict):
    with open(GICS_CACHE_PATH, 'w') as f:
        json.dump(cache, f)


def get_gics_for_tickers(tickers: list) -> pd.DataFrame:
    """
    Fetch GICS sector/industry for each ticker via yfinance.
    Results are cached to data/raw/yfinance_gics_cache.json.
    ASSUMPTION A3: Uses current GICS classification (not point-in-time).
    """
    cache = _load_gics_cache()
    uncached = [t for t in tickers if t not in cache]

    if uncached:
        print(f'Fetching GICS classification for {len(uncached)} uncached tickers...')
        for ticker in tqdm(uncached, desc='yfinance GICS'):
            try:
                info = yf.Ticker(ticker).info
                cache[ticker] = {
                    'gics_sector':       info.get('sector', 'Unknown'),
                    'gics_industry':     info.get('industry', 'Unknown'),
                    'gics_sub_industry': info.get('industryDisp', info.get('industry', 'Unknown')),
                    'fetched_date':      datetime.now().isoformat()[:10],
                }
            except Exception:
                cache[ticker] = {
                    'gics_sector': 'Unknown', 'gics_industry': 'Unknown',
                    'gics_sub_industry': 'Unknown', 'fetched_date': 'error',
                }
        _save_gics_cache(cache)

    rows = []
    for ticker in tickers:
        r = cache.get(ticker, {})
        rows.append({
            'Ticker':           ticker,
            'gics_sector':      r.get('gics_sector', 'Unknown'),
            'gics_industry':    r.get('gics_industry', 'Unknown'),
            'gics_sub_industry':r.get('gics_sub_industry', 'Unknown'),
        })
    return pd.DataFrame(rows)


def map_to_project_sector(gics_df: pd.DataFrame) -> pd.DataFrame:
    """
    Map GICS industry to one of the 7 project sectors using keyword matching.
    Tickers matching no sector are labeled 'Other' and excluded from signal engine.
    """
    df = gics_df.copy()
    df['project_sector'] = 'Other'

    for sector, cfg in SECTOR_CONFIG.items():
        keywords = cfg['gics_industry_keywords']
        kw_match = df['gics_industry'].apply(
            lambda x: any(kw.lower() in str(x).lower() for kw in keywords)
        )
        # Only assign if not yet mapped
        df.loc[kw_match & (df['project_sector'] == 'Other'), 'project_sector'] = sector

    return df


# ─── Run Sector Mapping ───
print('Mapping tickers to GICS project sectors...')
unique_tickers = df_raw['Ticker'].dropna().unique().tolist()
gics_df       = get_gics_for_tickers(unique_tickers)
ticker_sector_map = map_to_project_sector(gics_df)
ticker_sector_map = ticker_sector_map.astype(str)
import importlib
import sys
import pyarrow as pa

# Avoid duplicate Arrow extension registration in reused notebook kernels.
if 'pandas.core.arrays.arrow.extension_types' not in sys.modules:
    for ext_name in ('pandas.interval', 'pandas.period'):
        try:
            pa.unregister_extension_type(ext_name)
        except pa.ArrowKeyError:
            pass
    importlib.import_module('pandas.core.arrays.arrow.extension_types')
ticker_sector_map.to_parquet(
    PROCESSED_DIR / 'ticker_sector_map.parquet',
    index=False,
    engine='pyarrow',
    coerce_timestamps='ms',
    version='2.6',
    use_deprecated_int96_timestamps=False,
    use_dictionary=False,
)

print('\nProject sector distribution (top tickers):')
print(ticker_sector_map['project_sector'].value_counts())

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 3: Committee Mapper — Legislator → Committee Assignment
# ═══════════════════════════════════════════════════════════════════════════

# ASSUMPTION A4: Committee assignments matched at Congress session level,
# not exact date. Mid-session changes are rare; document in assumptions.md.
CONGRESS_SESSIONS = {
    114: ('2015-01-03', '2017-01-03'),
    115: ('2017-01-03', '2019-01-03'),
    116: ('2019-01-03', '2021-01-03'),
    117: ('2021-01-03', '2023-01-03'),
    118: ('2023-01-03', '2025-01-03'),
    119: ('2025-01-03', '2027-01-03'),
}

# Official committee name → config short name mapping
COMMITTEE_NAME_MAP = {
    'House Committee on Armed Services':           'House Armed Services',
    'Senate Committee on Armed Services':          'Senate Armed Services',
    'House Committee on Appropriations':           'House Appropriations',
    'Senate Committee on Appropriations':          'Senate Appropriations',
    'House Committee on Foreign Affairs':          'House Foreign Affairs',
    'Senate Committee on Foreign Relations':       'Senate Foreign Relations',
    'House Committee on Energy and Commerce':      'House Energy and Commerce',
    'Senate Committee on Health, Education, Labor, and Pensions': 'Senate HELP',
    'House Committee on Ways and Means':           'House Ways and Means',
    'Senate Committee on Finance':                 'Senate Finance',
    'House Committee on Science, Space, and Technology': 'House Science, Space, and Technology',
    'Senate Committee on Commerce, Science, and Transportation': 'Senate Commerce',
    'House Committee on the Judiciary':            'House Judiciary',
    'Senate Committee on the Judiciary':           'Senate Judiciary',
    'House Committee on Financial Services':       'House Financial Services',
    'Senate Committee on Banking, Housing, and Urban Affairs': 'Senate Banking',
    'Senate Committee on Energy and Natural Resources': 'Senate Energy and Natural Resources',
    'House Committee on Natural Resources':        'House Natural Resources',
    'Senate Committee on Environment and Public Works': 'Senate Environment and Public Works',
    'House Committee on Transportation and Infrastructure': 'House Transportation and Infrastructure',
}


def get_congress_session(trade_date) -> int:
    """Return Congress session number for a given date."""
    ts = pd.Timestamp(trade_date)
    for session, (start, end) in CONGRESS_SESSIONS.items():
        if pd.Timestamp(start) <= ts < pd.Timestamp(end):
            return session
    return 119  # default to current


def _get_congress_api_key() -> str:
    """Load CONGRESS_API_KEY from env, then .env fallback."""
    key = (os.environ.get('CONGRESS_API_KEY') or '').strip()
    if key:
        return key

    env_path = Path('.env')
    if env_path.exists():
        for raw in env_path.read_text().splitlines():
            line = raw.strip()
            if not line or line.startswith('#') or '=' not in line:
                continue
            k, v = line.split('=', 1)
            if k.strip() == 'CONGRESS_API_KEY':
                key = v.strip().strip('"').strip("'")
                if key:
                    os.environ['CONGRESS_API_KEY'] = key
                    return key
    return ''


def _append_api_key(url: str, api_key: str) -> str:
    """Attach api_key + format=json to congress.gov URLs if missing."""
    out = url
    if 'api_key=' not in out:
        out += ('&' if '?' in out else '?') + f'api_key={api_key}'
    if 'format=' not in out:
        out += '&format=json'
    return out


def _normalize_text(value: str) -> str:
    """Lowercase and keep alphanumeric tokens only."""
    chars = []
    for ch in str(value).lower():
        chars.append(ch if ch.isalnum() else ' ')
    return ' '.join(''.join(chars).split())


def _normalize_chamber(chamber: str) -> str:
    c = _normalize_text(chamber)
    if 'house' in c:
        return 'house'
    if 'senate' in c:
        return 'senate'
    return ''


TARGET_COMMITTEE_RULES = [
    {'chamber': 'house',  'keywords': ['armed', 'services'],                         'mapped': 'House Armed Services'},
    {'chamber': 'senate', 'keywords': ['armed', 'services'],                         'mapped': 'Senate Armed Services'},
    {'chamber': 'house',  'keywords': ['appropriations'],                            'mapped': 'House Appropriations'},
    {'chamber': 'senate', 'keywords': ['appropriations'],                            'mapped': 'Senate Appropriations'},
    {'chamber': 'house',  'keywords': ['foreign', 'affairs'],                        'mapped': 'House Foreign Affairs'},
    {'chamber': 'senate', 'keywords': ['foreign', 'relations'],                      'mapped': 'Senate Foreign Relations'},
    {'chamber': 'house',  'keywords': ['energy', 'commerce'],                        'mapped': 'House Energy and Commerce'},
    {'chamber': 'senate', 'keywords': ['health', 'education', 'labor', 'pensions'], 'mapped': 'Senate HELP'},
    {'chamber': 'house',  'keywords': ['ways', 'means'],                             'mapped': 'House Ways and Means'},
    {'chamber': 'senate', 'keywords': ['finance'],                                   'mapped': 'Senate Finance'},
    {'chamber': 'house',  'keywords': ['science', 'space', 'technology'],            'mapped': 'House Science, Space, and Technology'},
    {'chamber': 'senate', 'keywords': ['commerce', 'science', 'transportation'],     'mapped': 'Senate Commerce'},
    {'chamber': 'house',  'keywords': ['judiciary'],                                 'mapped': 'House Judiciary'},
    {'chamber': 'senate', 'keywords': ['judiciary'],                                 'mapped': 'Senate Judiciary'},
    {'chamber': 'house',  'keywords': ['financial', 'services'],                     'mapped': 'House Financial Services'},
    {'chamber': 'senate', 'keywords': ['banking', 'housing', 'urban', 'affairs'],   'mapped': 'Senate Banking'},
    {'chamber': 'senate', 'keywords': ['energy', 'natural', 'resources'],            'mapped': 'Senate Energy and Natural Resources'},
    {'chamber': 'house',  'keywords': ['natural', 'resources'],                      'mapped': 'House Natural Resources'},
    {'chamber': 'senate', 'keywords': ['environment', 'public', 'works'],            'mapped': 'Senate Environment and Public Works'},
    {'chamber': 'house',  'keywords': ['transportation', 'infrastructure'],          'mapped': 'House Transportation and Infrastructure'},
]


def _map_committee_name(raw_name: str, chamber: str) -> str | None:
    """Map congress.gov committee names to project committee labels."""
    exact = COMMITTEE_NAME_MAP.get(raw_name)
    if exact:
        return exact

    name_norm = _normalize_text(raw_name)
    chamber_norm = _normalize_chamber(chamber)

    # Project config is keyed to full committees, not subcommittees.
    if 'subcommittee' in name_norm:
        return None

    for rule in TARGET_COMMITTEE_RULES:
        if chamber_norm != rule['chamber']:
            continue
        if all(keyword in name_norm for keyword in rule['keywords']):
            return rule['mapped']
    return None




_COMMITTEE_ROSTER_FALLBACK_CACHE = None


def _get_open_data_committee_roster() -> pd.DataFrame:
    """
    Build fallback roster from unitedstates/congress-legislators current committee data.
    Used when Congress API omits committee membership lists.
    """
    global _COMMITTEE_ROSTER_FALLBACK_CACHE
    if _COMMITTEE_ROSTER_FALLBACK_CACHE is not None:
        return _COMMITTEE_ROSTER_FALLBACK_CACHE.copy()

    base_url = 'https://raw.githubusercontent.com/unitedstates/congress-legislators/gh-pages'
    try:
        committees_resp = requests.get(f'{base_url}/committees-current.json', timeout=30)
        members_resp = requests.get(f'{base_url}/committee-membership-current.json', timeout=30)
        legislators_resp = requests.get(f'{base_url}/legislators-current.json', timeout=30)
    except Exception as e:
        print(f'  Open-data fallback fetch error: {e}')
        return pd.DataFrame()

    if (
        committees_resp.status_code != 200
        or members_resp.status_code != 200
        or legislators_resp.status_code != 200
    ):
        return pd.DataFrame()

    committees = committees_resp.json()
    committee_membership = members_resp.json()
    legislators = legislators_resp.json()

    bioguide_to_name = {}
    for leg in legislators:
        leg_id = (leg.get('id') or {}).get('bioguide')
        name_info = leg.get('name') or {}
        display_name = name_info.get('official_full')
        if not display_name:
            display_name = ' '.join(
                part
                for part in [name_info.get('first'), name_info.get('middle'), name_info.get('last')]
                if part
            ).strip()
        if leg_id and display_name:
            bioguide_to_name[leg_id] = display_name

    rows = []
    for cmte in committees:
        committee_id = cmte.get('thomas_id')
        if not committee_id:
            continue

        chamber_type = str(cmte.get('type', '')).lower()
        chamber = 'House' if chamber_type == 'house' else 'Senate' if chamber_type == 'senate' else str(cmte.get('chamber', ''))

        mapped_name = _map_committee_name(cmte.get('name', ''), chamber)
        if not mapped_name:
            continue

        members = committee_membership.get(committee_id, [])
        for member in members:
            member_name = str(member.get('name') or '').strip()
            if not member_name:
                member_name = bioguide_to_name.get(member.get('bioguide'), '')
            if not member_name:
                continue
            rows.append({
                'member_name': member_name,
                'committee_name': mapped_name,
                'chamber': chamber,
            })

    fallback = pd.DataFrame(rows)
    if not fallback.empty:
        fallback = fallback.drop_duplicates().reset_index(drop=True)

    _COMMITTEE_ROSTER_FALLBACK_CACHE = fallback
    return fallback.copy()


def _extract_member_names(payload: dict) -> list:
    """Recursively extract member names from congress API payloads."""
    names = []

    def walk(node):
        if isinstance(node, dict):
            for key, value in node.items():
                lk = str(key).lower()
                if lk in {'members', 'member', 'committee_members', 'committeemembers'} and isinstance(value, list):
                    for item in value:
                        if isinstance(item, str):
                            names.append(item)
                        elif isinstance(item, dict):
                            name = (
                                item.get('name')
                                or item.get('memberName')
                                or item.get('fullName')
                                or (item.get('member', {}) or {}).get('name')
                            )
                            if name:
                                names.append(str(name))
                elif isinstance(value, (dict, list)):
                    walk(value)
        elif isinstance(node, list):
            for item in node:
                if isinstance(item, (dict, list)):
                    walk(item)

    walk(payload)
    deduped = []
    seen = set()
    for name in names:
        n = str(name).strip()
        if n and n not in seen:
            seen.add(n)
            deduped.append(n)
    return deduped


def fetch_committee_rosters_from_api(congress_session: int) -> pd.DataFrame:
    """
    Fetch committee rosters from congress.gov API.
    Falls back to committee detail URLs when member lists are not inline.
    """
    api_key = _get_congress_api_key()
    if not api_key:
        if congress_session == min(CONGRESS_SESSIONS):
            print('  CONGRESS_API_KEY missing in environment and .env.')
        return pd.DataFrame()

    url = (f'https://api.congress.gov/v3/committee?congress={congress_session}'
           f'&api_key={api_key}&format=json&limit=250')

    results = []
    try:
        resp = requests.get(url, timeout=30)
        if resp.status_code != 200:
            print(f'  Congress API returned HTTP {resp.status_code} for session {congress_session}.')
            return pd.DataFrame()

        payload = resp.json()
        committees = payload.get('committees', [])

        target_committees = []
        for cmte in committees:
            mapped_name = _map_committee_name(cmte.get('name', ''), cmte.get('chamber', ''))
            if mapped_name:
                cmte_copy = dict(cmte)
                cmte_copy['_mapped_name'] = mapped_name
                target_committees.append(cmte_copy)

        if congress_session == min(CONGRESS_SESSIONS):
            print(f'  Committees returned: {len(committees)} total; {len(target_committees)} mapped targets')
            if target_committees:
                sample_mapped = [(c.get('name', 'Unknown'), c.get('_mapped_name')) for c in target_committees[:5]]
                print(f'  Sample mapped committees: {sample_mapped}')
            elif committees:
                sample_names = [c.get('name', 'Unknown') for c in committees[:5]]
                print(f'  Sample committee names: {sample_names}')

        for cmte in target_committees:
            raw_name = cmte.get('name', '')
            cmte_name = cmte.get('_mapped_name', raw_name)
            chamber = cmte.get('chamber', '')

            member_names = _extract_member_names(cmte)

            if not member_names:
                detail_urls = []
                for key in ('url', 'api_uri'):
                    if cmte.get(key):
                        detail_urls.append(cmte.get(key))

                members_field = cmte.get('members')
                if isinstance(members_field, dict):
                    for key in ('url', 'api_uri'):
                        if members_field.get(key):
                            detail_urls.append(members_field.get(key))
                elif isinstance(members_field, str):
                    detail_urls.append(members_field)

                for detail_url in detail_urls:
                    try:
                        dresp = requests.get(_append_api_key(detail_url, api_key), timeout=30)
                        if dresp.status_code == 200:
                            member_names = _extract_member_names(dresp.json())
                            if member_names:
                                break
                        elif congress_session == min(CONGRESS_SESSIONS):
                            print(f'  Detail fetch HTTP {dresp.status_code} for {raw_name}.')
                    except Exception as detail_err:
                        if congress_session == min(CONGRESS_SESSIONS):
                            print(f'  Detail fetch failed for {raw_name}: {detail_err}')

            for member_name in member_names:
                results.append({
                    'member_name': member_name,
                    'committee_name': cmte_name,
                    'congress_session': congress_session,
                    'chamber': chamber,
                })

        if congress_session == min(CONGRESS_SESSIONS):
            print(f'  Extracted assignments from API payload: {len(results):,}')

    except Exception as e:
        print(f'  Congress API error for session {congress_session}: {e}')

    if results:
        return pd.DataFrame(results).drop_duplicates().reset_index(drop=True)

    fallback_df = _get_open_data_committee_roster()
    if not fallback_df.empty:
        fallback_df = fallback_df.copy()
        fallback_df['congress_session'] = congress_session
        if congress_session == min(CONGRESS_SESSIONS):
            print('  Congress API does not expose committee membership; using open-data fallback roster.')
            print('  Fallback assumption: current committee assignments are projected across sessions.')
            print(f'  Open-data fallback assignments: {len(fallback_df):,}')
        return fallback_df

    return pd.DataFrame()


def assign_committees_to_trades(trades_df: pd.DataFrame,
                                roster_df: pd.DataFrame) -> pd.DataFrame:
    """
    Tag each trade with Is_Committee_Relevant.

    If roster is available: fuzzy-match legislator names (threshold >= 85),
    look up committee assignments at time of trade, check against SECTOR_CONFIG.
    If roster is empty: conservative fallback — all False (logged as warning).
    """
    df = trades_df.copy()
    if 'TransactionDate' in df.columns:
        df['Congress_Session'] = df['TransactionDate'].apply(get_congress_session)
    else:
        df['Congress_Session'] = 119

    if roster_df.empty or 'member_name' not in roster_df.columns:
        df['Committee_List']        = [[] for _ in range(len(df))]
        df['Is_Committee_Relevant'] = False
        print('WARNING: No committee roster available. All trades marked non-committee-relevant.')
        print('         Set CONGRESS_API_KEY to enable H2 analysis.')
        return df

    # Build name lookup via fuzzy matching
    roster_names = roster_df['member_name'].str.lower().str.strip().unique().tolist()
    name_cache = {}
    for name in df['Name'].str.lower().str.strip().unique():
        if fuzz_process:
            match, score = fuzz_process.extractOne(name, roster_names,
                                                   scorer=fuzz.token_sort_ratio)
            name_cache[name] = match if score >= 85 else None
        else:
            name_cache[name] = None

    def get_committees(row):
        matched = name_cache.get(str(row['Name']).lower().strip())
        if matched is None:
            return []
        mask = (
            (roster_df['member_name'].str.lower().str.strip() == matched) &
            (roster_df['congress_session'] == row['Congress_Session'])
        )
        return roster_df.loc[mask, 'committee_name'].tolist()

    def is_relevant(row):
        sector = row.get('project_sector', 'Other')
        if sector == 'Other' or sector not in SECTOR_CONFIG:
            return False
        sector_committees = set(SECTOR_CONFIG[sector]['committees'])
        return bool(sector_committees & set(row.get('Committee_List', [])))

    df['Committee_List']        = df.apply(get_committees, axis=1)
    df['Is_Committee_Relevant'] = df.apply(is_relevant, axis=1)
    return df


# ─── Run Committee Mapping ───
print('Fetching committee rosters...')
roster_frames = []
for session in CONGRESS_SESSIONS:
    r = fetch_committee_rosters_from_api(session)
    if not r.empty:
        roster_frames.append(r)

if roster_frames:
    committee_roster = pd.concat(roster_frames, ignore_index=True)
    committee_roster.to_parquet(PROCESSED_DIR / 'committee_roster.parquet', index=False)
    print(f'Committee roster loaded: {len(committee_roster):,} assignments')
else:
    committee_roster = pd.DataFrame()
    print('No committee roster from API — using fallback (Is_Committee_Relevant = False).')
    print('Set CONGRESS_API_KEY env variable to enable full committee analysis.')

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULES 4-6: Stock Returns, ETF Returns, Fama-French Factors
# ═══════════════════════════════════════════════════════════════════════════

# ─── Module 4: Stock Returns ───
def fetch_stock_returns(tickers: list, start: str, end: str) -> pd.DataFrame:
    """
    Fetch daily adjusted-close returns for event study.
    Returns computed as (adj_close_t / adj_close_{t-1}) - 1.
    Caches to data/raw/stock_returns_cache.parquet.
    """
    cache_path = RAW_DIR / 'stock_returns_cache.parquet'

    if cache_path.exists():
        cached = pd.read_parquet(cache_path)
        cached_tickers = set(cached['Ticker'].unique())
        need_tickers = [t for t in tickers if t not in cached_tickers]
    else:
        cached = pd.DataFrame()
        need_tickers = tickers

    if yf is None:
        print('yfinance not available; using cached stock returns if present.')
        return cached

    if need_tickers:
        print(f'Fetching returns for {len(need_tickers)} new tickers...')
        frames = []
        batch_size = 50
        for i in tqdm(range(0, len(need_tickers), batch_size), desc='Stock returns'):
            batch = need_tickers[i:i + batch_size]
            try:
                prices = yf.download(batch, start=start, end=end,
                                     auto_adjust=True, progress=False)['Close']
                if isinstance(prices, pd.Series):
                    prices = prices.to_frame(batch[0])
                ret = prices.pct_change().dropna(how='all')
                df_long = (
                    ret.reset_index()
                       .melt(id_vars='Date', var_name='Ticker', value_name='ret')
                       .rename(columns={'Date': 'date'})
                       .dropna(subset=['ret'])
                )
                frames.append(df_long)
            except Exception as e:
                print(f'  Batch {i}: {e}')

        if frames:
            new_data = pd.concat(frames, ignore_index=True)
            new_data['date'] = pd.to_datetime(new_data['date'])
            stock_returns = (
                pd.concat([cached, new_data], ignore_index=True)
                  .drop_duplicates(subset=['date', 'Ticker'])
            )
            stock_returns.to_parquet(cache_path, index=False)
        else:
            stock_returns = cached
    else:
        stock_returns = cached
        print(f'All {len(tickers)} tickers found in cache.')

    stock_returns['date'] = pd.to_datetime(stock_returns['date'])
    return stock_returns

# ─── Module 5: ETF Returns ───
def fetch_etf_returns(etf_tickers: list, start: str, end: str) -> pd.DataFrame:
    """Fetch daily returns for sector ETFs + SPY benchmark."""
    all_etfs = list(set(etf_tickers + ['SPY']))
    print(f'Fetching ETF returns for: {all_etfs}')
    if yf is None:
        print('yfinance not available; using cached ETF returns if present.')
        cache = EXTERNAL_DIR / 'etf_returns.parquet'
        return pd.read_parquet(cache) if cache.exists() else pd.DataFrame(columns=['date','etf_ticker','ret'])
    try:
        prices = yf.download(all_etfs, start=start, end=end,
                             auto_adjust=True, progress=False)['Close']
        if isinstance(prices, pd.Series):
            prices = prices.to_frame(all_etfs[0])
        ret = prices.pct_change().dropna(how='all')
        df_long = (
            ret.reset_index()
               .melt(id_vars='Date', var_name='etf_ticker', value_name='ret')
               .rename(columns={'Date': 'date'})
        )
        df_long['date'] = pd.to_datetime(df_long['date'])
        df_long.to_parquet(EXTERNAL_DIR / 'etf_returns.parquet', index=False)
        print(f'ETF returns: {len(df_long):,} daily observations')
        return df_long
    except Exception as e:
        print(f'ETF fetch failed: {e}')
        return pd.DataFrame(columns=['date', 'etf_ticker', 'ret'])


# ─── Module 6: Fama-French Factors ───
def fetch_ff5_mom(start: str, end: str) -> pd.DataFrame:
    """
    Download FF5 + Momentum daily factors from Kenneth French Data Library.
    Values are in % — divided by 100 for decimal returns.
    COLUMN NAMES standardized: Mkt-RF → Mkt_RF, Mom column normalized.
    """
    ff_path = EXTERNAL_DIR / 'ff5_mom_factors.csv'
    if ff_path.exists():
        df = pd.read_csv(ff_path, index_col='date', parse_dates=True)
        print(f'FF5+Mom factors loaded from cache: {len(df):,} rows')
        return df

    global web
    if web is None:
        try:
            import pandas_datareader.data as web_retry
            web = web_retry
            print('pandas_datareader loaded at runtime.')
        except ImportError:
            print('pandas_datareader not installed — skipping factor download.')
            return pd.DataFrame()

    print('Downloading Fama-French 5 + Momentum factors (daily)...')
    try:
        ff5 = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily',
                             'famafrench', start=start, end=end)[0]
        mom = web.DataReader('F-F_Momentum_Factor_daily',
                             'famafrench', start=start, end=end)[0]
        ff5.index = pd.to_datetime(ff5.index, format='%Y%m%d')
        mom.index = pd.to_datetime(mom.index, format='%Y%m%d')
        df = ff5.join(mom, how='inner') / 100.0
        df.columns = [c.strip().replace('-', '_').replace(' ', '') for c in df.columns]
        # Normalize momentum column name
        mom_col = next((c for c in df.columns if 'mom' in c.lower() or c == 'WML'), None)
        if mom_col and mom_col != 'Mom':
            df = df.rename(columns={mom_col: 'Mom'})
        df.index.name = 'date'
        df.to_csv(ff_path)
        print(f'FF5+Mom factors: {len(df):,} daily observations')
        return df
    except Exception as e:
        print(f'Daily factors failed ({e}), trying monthly...')
        try:
            ff5 = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                                 'famafrench', start=start, end=end)[0]
            mom = web.DataReader('F-F_Momentum_Factor',
                                 'famafrench', start=start, end=end)[0]
            ff5.index = pd.to_datetime(ff5.index.to_timestamp())
            mom.index = pd.to_datetime(mom.index.to_timestamp())
            df = ff5.join(mom, how='inner') / 100.0
            df.columns = [c.strip().replace('-', '_').replace(' ', '') for c in df.columns]
            mom_col = next((c for c in df.columns if 'mom' in c.lower() or c == 'WML'), None)
            if mom_col and mom_col != 'Mom':
                df = df.rename(columns={mom_col: 'Mom'})
            df.index.name = 'date'
            df.to_csv(ff_path)
            print(f'FF5+Mom monthly factors: {len(df):,} rows')
            return df
        except Exception as e2:
            print(f'Factor download unavailable: {e2}')
            return pd.DataFrame()


# ─── Fetch all return data ───
etf_tickers   = [cfg['target_etf'] for cfg in SECTOR_CONFIG.values()]
etf_returns   = fetch_etf_returns(etf_tickers, ESTIMATION_WINDOW_START, SAMPLE_END)
ff_factors    = fetch_ff5_mom(ESTIMATION_WINDOW_START, SAMPLE_END)

# Market returns from SPY
spy_daily = (
    etf_returns[etf_returns['etf_ticker'] == 'SPY']
    .set_index('date')['ret']
    .rename('mkt_ret')
)

print(f'\nData fetch complete:')
print(f'  ETF returns: {len(etf_returns):,} rows')
print(f'  Market (SPY) returns: {len(spy_daily):,} days')
print(f'  FF5+Mom factors: {len(ff_factors):,} rows')

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 7: Enrichment Pipeline — Master Join
# ═══════════════════════════════════════════════════════════════════════════

def run_enrichment_pipeline(df_raw: pd.DataFrame,
                            ticker_sector_map: pd.DataFrame,
                            committee_roster: pd.DataFrame) -> pd.DataFrame:
    """
    Master join: trades → GICS sectors → project sectors → committee flags → ETF mapping.
    Validates final schema and logs counts to console.
    Saves to data/processed/trades_enriched.parquet.
    """
    df = df_raw.copy()
    n_raw = len(df)

    # Step 1: Join sector mapping
    df = df.merge(
        ticker_sector_map[['Ticker', 'gics_sector', 'gics_industry', 'project_sector']],
        on='Ticker', how='left'
    )
    df['project_sector'] = df['project_sector'].fillna('Other')
    n_sector_matched = (df['project_sector'] != 'Other').sum()

    # Step 2: Add target_etf
    sector_to_etf = {k: v['target_etf'] for k, v in SECTOR_CONFIG.items()}
    df['target_etf'] = df['project_sector'].map(sector_to_etf)

    # Step 3: Assign committee memberships
    df = assign_committees_to_trades(df, committee_roster)
    n_committee_relevant = df['Is_Committee_Relevant'].sum()

    # Step 4: Ensure all required columns exist
    required = [
        'ReportDate', 'TransactionDate', 'Ticker', 'Name',
        'Transaction', 'Direction', 'Amount', 'AmountMidpoint',
        'DisclosureLag', 'Chamber', 'gics_sector', 'gics_industry',
        'project_sector', 'Committee_List', 'Is_Committee_Relevant', 'target_etf',
    ]
    for col in required:
        if col not in df.columns:
            df[col] = np.nan if col not in ['Committee_List'] else [[] for _ in range(len(df))]

    df.to_parquet(PROCESSED_DIR / 'trades_enriched.parquet', index=False)

    # Validation report
    print('\n' + '=' * 60)
    print('ENRICHMENT VALIDATION REPORT')
    print('=' * 60)
    print(f'  Total raw trades:              {n_raw:>10,}')
    print(f'  Matched to project sector:     {n_sector_matched:>10,} ({n_sector_matched/n_raw*100:.1f}%)')
    print(f'  "Other" sector (excluded):     {(df["project_sector"]=="Other").sum():>10,}')
    print(f'  Committee-relevant trades:     {n_committee_relevant:>10,} ({n_committee_relevant/n_raw*100:.1f}%)')
    print(f'  Final enriched dataset rows:   {len(df):>10,}')
    print('=' * 60)

    return df


df_enriched = run_enrichment_pipeline(df_raw, ticker_sector_map, committee_roster)
df_enriched.head()

---
### Data Exploration & Quality Assessment

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Congressional Trading Data — Exploratory Analysis', fontsize=16, fontweight='bold')

# 1. Monthly trade disclosures over time
ax = axes[0, 0]
monthly_trades = df_enriched.set_index('ReportDate').resample('ME').size()
ax.plot(monthly_trades.index, monthly_trades.values, linewidth=1.5, color='steelblue')
ax.set_title('Monthly Trade Disclosures Over Time')
ax.set_xlabel('Report Date')
ax.set_ylabel('Number of Trades')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# 2. Purchase vs. Sale
ax = axes[0, 1]
dir_counts = df_enriched['Transaction'].value_counts()
bars = ax.bar(dir_counts.index, dir_counts.values,
              color=['steelblue', 'coral'], edgecolor='white')
ax.set_title('Purchase vs. Sale Distribution')
ax.set_ylabel('Count')
for bar, val in zip(bars, dir_counts.values):
    ax.text(bar.get_x() + bar.get_width() / 2., bar.get_height() * 1.02,
            f'{val:,}', ha='center', fontsize=10)

# 3. Project sector breakdown
ax = axes[0, 2]
sector_counts = df_enriched['project_sector'].value_counts()
colors = plt.cm.Set3(np.linspace(0, 1, len(sector_counts)))
ax.barh(sector_counts.index, sector_counts.values, color=colors)
ax.set_title('Trades by Project Sector')
ax.set_xlabel('Count')

# 4. Disclosure lag
ax = axes[1, 0]
lags = df_enriched['DisclosureLag'].dropna().clip(0, 120)
ax.hist(lags, bins=60, color='steelblue', alpha=0.7, edgecolor='white')
ax.axvline(45, color='red', linestyle='--', label='45-day STOCK Act deadline')
ax.set_title('Disclosure Lag Distribution')
ax.set_xlabel('Calendar Days (ReportDate − TransactionDate)')
ax.set_ylabel('Frequency')
ax.legend()

# 5. Amount distribution
ax = axes[1, 1]
amounts = df_enriched['AmountMidpoint'].dropna()
amounts_pos = amounts[amounts > 0]
ax.hist(np.log10(amounts_pos), bins=50, color='mediumpurple', alpha=0.7, edgecolor='white')
ax.set_title('Trade Amount Distribution (log₁₀ scale)')
ax.set_xlabel('log₁₀(AmountMidpoint USD)')
ax.set_ylabel('Frequency')

# 6. Committee relevance pie
ax = axes[1, 2]
comm_counts = (
    df_enriched['Is_Committee_Relevant']
    .fillna(False)
    .astype(bool)
    .value_counts()
    .reindex([False, True], fill_value=0)
)
labels = ['Non-Relevant', 'Committee-Relevant']
if comm_counts.sum() == 0:
    ax.text(0.5, 0.5, 'No committee data', ha='center', va='center')
    ax.axis('off')
else:
    ax.pie(comm_counts.values, labels=labels, colors=['lightcoral', 'steelblue'],
           autopct='%1.1f%%', startangle=90)
ax.set_title('Committee Relevance of Trades')

plt.tight_layout()
plt.show()

print('\n' + '=' * 60)
print('DATA QUALITY SUMMARY')
print('=' * 60)
print(f'Unique legislators: {df_enriched["Name"].nunique():,}')
print(f'Unique tickers:     {df_enriched["Ticker"].nunique():,}')
print(f'Date range:         {df_enriched["ReportDate"].min().date()} → {df_enriched["ReportDate"].max().date()}')
if 'DisclosureLag' in df_enriched.columns:
    print(f'Median lag (days):  {df_enriched["DisclosureLag"].median():.0f}')
    print(f'Lag > 45 days:      {(df_enriched["DisclosureLag"] > 45).sum():,} '
          f'({(df_enriched["DisclosureLag"] > 45).mean()*100:.1f}%)')
print(f'Purchases:          {(df_enriched["Direction"] == 1).sum():,}')
print(f'Sales:              {(df_enriched["Direction"] == -1).sum():,}')

---
## Phase 2: H1 — Market Underreaction Event Study

**Hypothesis:** If `CAR[-1,+1] ≈ 0` but `CAR[+2,+20] > 0` for purchases after disclosure, the market underreacts to congressional trade disclosures.

**Event date = `ReportDate` (disclosure date) — NEVER `TransactionDate`.**

**Definition of Done:**
- CAR[-1,+1] and CAR[+2,+20] computed for all disclosure events
- Cross-sectional t-test with buy/sell separation and Patell standardized test
- Clear accept/reject conclusion for H1
- Robustness: winsorized, lag ≤ 45 days, subperiod splits

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 8: Market Model Estimation
# ═══════════════════════════════════════════════════════════════════════════

# Fetch stock returns for all unique tickers in the trade dataset
print('Fetching stock returns for event study...')
unique_tickers = df_enriched['Ticker'].dropna().unique().tolist()
stock_returns_df = fetch_stock_returns(unique_tickers, ESTIMATION_WINDOW_START, SAMPLE_END)

# Build wide returns matrix (date × ticker) for fast indexing
stock_rets = stock_returns_df.pivot(index='date', columns='Ticker', values='ret')
stock_rets.index = pd.to_datetime(stock_rets.index)
stock_rets = stock_rets.sort_index()

print(f'Return matrix: {stock_rets.shape[0]:,} dates × {stock_rets.shape[1]:,} tickers')


def fit_market_model(event_date: pd.Timestamp,
                    ticker: str,
                    stock_rets_matrix: pd.DataFrame,
                    mkt_returns: pd.Series,
                    estimation_window: tuple = (-250, -30),
                    min_obs: int = 120) -> dict:
    if sm is None:
        return None
    """
    OLS: stock_ret_t = alpha + beta * mkt_ret_t + epsilon
    Estimation window in TRADING DAYS relative to event_date.
    Returns None if insufficient data.
    """
    if ticker not in stock_rets_matrix.columns:
        return None

    all_dates = list(stock_rets_matrix.index)
    dates_before = [d for d in all_dates if d <= event_date]
    if not dates_before:
        return None

    event_idx = all_dates.index(dates_before[-1])
    start_idx = max(0, event_idx + estimation_window[0])
    end_idx   = max(0, event_idx + estimation_window[1])

    if end_idx <= start_idx:
        return None

    est_dates  = stock_rets_matrix.index[start_idx:end_idx]
    stock_r    = stock_rets_matrix.loc[est_dates, ticker].dropna()
    market_r   = mkt_returns.reindex(est_dates).dropna()
    common     = stock_r.index.intersection(market_r.index)

    if len(common) < min_obs:
        return None

    try:
        y = stock_r.loc[common].values
        X = sm.add_constant(market_r.loc[common].values)
        res = sm.OLS(y, X).fit()
        return {
            'alpha':     res.params[0],
            'beta':      res.params[1],
            'sigma':     res.resid.std(),
            'n_obs':     len(common),
            'r_squared': res.rsquared,
        }
    except Exception:
        return None


# ─── Aggregate to disclosure-level events ───
# CRITICAL: Event date = ReportDate (disclosure date)
events = (
    df_enriched
    .groupby(['ReportDate', 'Ticker', 'Direction'])
    .agg(TotalAmount=('AmountMidpoint', 'sum'), n_trades=('Direction', 'count'))
    .reset_index()
)
print(f'\nTotal disclosure events: {len(events):,}')
print(f'  Purchases: {(events["Direction"] == 1).sum():,}')
print(f'  Sales:     {(events["Direction"] == -1).sum():,}')

# ─── Fit market models ───
print('\nFitting market models (may take several minutes)...')
market_models = {}
failed = 0

for _, row in tqdm(events.iterrows(), total=len(events), desc='Market Models'):
    key = (pd.Timestamp(row['ReportDate']), row['Ticker'])
    result = fit_market_model(
        event_date=key[0], ticker=key[1],
        stock_rets_matrix=stock_rets,
        mkt_returns=spy_daily,
        estimation_window=ESTIMATION_WINDOW,
        min_obs=MIN_ESTIMATION_DAYS,
    )
    if result is not None:
        market_models[key] = result
    else:
        failed += 1

print(f'\nMarket models fitted:   {len(market_models):,}')
print(f'Events excluded:        {failed:,} ({failed/len(events)*100:.1f}%)')

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 9: Abnormal Returns & CAR Computation
# ═══════════════════════════════════════════════════════════════════════════

def compute_car(event_date: pd.Timestamp, ticker: str,
                mm: dict,
                stock_rets_matrix: pd.DataFrame,
                mkt_returns: pd.Series,
                window: tuple) -> tuple:
    """
    Compute CAR[window[0], window[1]] in trading days relative to event_date.
    AR_t = actual_ret_t − (alpha + beta × mkt_ret_t)

    Returns (car, n_days). If stock is delisted during window, CAR is truncated
    (NOT forward-filled). Returns (nan, 0) on failure.
    """
    if ticker not in stock_rets_matrix.columns:
        return np.nan, 0

    try:
        all_dates = list(stock_rets_matrix.index)
        dates_before = [d for d in all_dates if d <= event_date]
        if not dates_before:
            return np.nan, 0

        event_idx = all_dates.index(dates_before[-1])
        start_idx = max(0, event_idx + window[0])
        end_idx   = min(len(all_dates) - 1, event_idx + window[1])

        if start_idx > end_idx:
            return np.nan, 0

        win_dates = stock_rets_matrix.index[start_idx:end_idx + 1]
        stock_r   = stock_rets_matrix.loc[win_dates, ticker].dropna()
        market_r  = mkt_returns.reindex(win_dates).dropna()
        common    = stock_r.index.intersection(market_r.index)

        if len(common) == 0:
            return np.nan, 0

        ar  = stock_r.loc[common] - (mm['alpha'] + mm['beta'] * market_r.loc[common])
        return ar.sum(), len(common)
    except Exception:
        return np.nan, 0


# ─── Compute CARs for all fitted events ───
print('Computing CARs...')
car_rows = []

for _, row in tqdm(events.iterrows(), total=len(events), desc='CAR Computation'):
    key = (pd.Timestamp(row['ReportDate']), row['Ticker'])
    if key not in market_models:
        continue

    mm = market_models[key]
    car_imm,   n_imm   = compute_car(key[0], key[1], mm, stock_rets, spy_daily, EVENT_WINDOW_IMMEDIATE)
    car_drift, n_drift = compute_car(key[0], key[1], mm, stock_rets, spy_daily, EVENT_WINDOW_DRIFT)
    ar_day0,   _       = compute_car(key[0], key[1], mm, stock_rets, spy_daily, (0, 0))

    car_rows.append({
        'ReportDate':      row['ReportDate'],
        'Ticker':          row['Ticker'],
        'Direction':       row['Direction'],
        'Transaction':     'Purchase' if row['Direction'] == 1 else 'Sale',
        'TotalAmount':     row['TotalAmount'],
        'mm_alpha':        mm['alpha'],
        'mm_beta':         mm['beta'],
        'mm_sigma':        mm['sigma'],
        'mm_n_obs':        mm['n_obs'],
        'mm_r2':           mm['r_squared'],
        'CAR_immediate':   car_imm,
        'CAR_drift':       car_drift,
        'AR_day0':         ar_day0,
        'n_days_immediate':n_imm,
        'n_days_drift':    n_drift,
    })

df_cars = pd.DataFrame(car_rows).dropna(subset=['CAR_immediate', 'CAR_drift'])
print(f'\nEvents with valid CARs: {len(df_cars):,}')
print('\nCAR summary statistics:')
print(df_cars[['CAR_immediate', 'CAR_drift', 'AR_day0']].describe().round(4))

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 10: H1 Statistical Tests
# ═══════════════════════════════════════════════════════════════════════════

def run_h1_tests(cars_df: pd.DataFrame) -> dict:
    """
    Cross-sectional t-tests: H0: mean CAR = 0 for each (direction, window) combination.
    Also computes simplified Patell standardized test.
    Reports: t-stat, p-value, n, mean_car, pct_positive.
    """
    results = {}
    for direction, label in [(1, 'buys'), (-1, 'sells')]:
        subset = cars_df[cars_df['Direction'] == direction].copy()
        for window_name, col, n_days_col in [
            ('immediate', 'CAR_immediate', 'n_days_immediate'),
            ('drift',     'CAR_drift',     'n_days_drift'),
        ]:
            cars = subset[col].dropna()
            key  = f'{label}_{window_name}'
            n    = len(cars)
            if n < 10:
                results[key] = {'mean_car': np.nan, 't_stat': np.nan,
                                'p_value': np.nan, 'n': n,
                                'pct_positive': np.nan, 't_patell': np.nan}
                continue

            t_stat, p_value = stats.ttest_1samp(cars, 0)

            # Simplified Patell (1976) standardized test
            sigma = subset.loc[cars.index, 'mm_sigma']
            n_obs = subset.loc[cars.index, 'mm_n_obs']
            n_win = subset.loc[cars.index, n_days_col].clip(lower=1)
            std_cars = cars / (sigma * np.sqrt(n_win) + 1e-12)
            t_patell = std_cars.mean() * np.sqrt(n) if n > 0 else np.nan

            results[key] = {
                'mean_car':    cars.mean(),
                'median_car':  cars.median(),
                't_stat':      t_stat,
                'p_value':     p_value,
                't_patell':    t_patell,
                'n':           n,
                'pct_positive':  (cars > 0).mean(),
            }
    return results


h1_results = run_h1_tests(df_cars)

print('\n' + '=' * 80)
print('H1 EVENT STUDY — Market Underreaction to Congressional Trade Disclosures')
print('=' * 80)

for key, res in h1_results.items():
    label, window = key.split('_', 1)
    win_str = 'CAR[-1,+1]' if window == 'immediate' else 'CAR[+2,+20]'
    stars = ''
    if not np.isnan(res.get('p_value', np.nan)):
        p = res['p_value']
        stars = '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

    print(f'\n{label.upper()} — {win_str}:')
    print(f'  Mean CAR:    {res["mean_car"]*100 if not np.isnan(res.get("mean_car",np.nan)) else "N/A":>8} %{stars}')
    print(f'  t-statistic: {res.get("t_stat",np.nan):>8.3f}')
    print(f'  p-value:     {res.get("p_value",np.nan):>8.4f}')
    print(f'  Patell t:    {res.get("t_patell",np.nan):>8.3f}')
    print(f'  N events:    {res.get("n",0):>8,}')
    print(f'  % Positive:  {res.get("pct_positive",np.nan)*100 if not np.isnan(res.get("pct_positive",np.nan)) else "N/A":>7.1f} %')

# Interpretation
print('\n' + '=' * 80)
print('H1 INTERPRETATION')
print('=' * 80)
bi_p = h1_results.get('buys_immediate', {}).get('p_value', 1)
bd_p = h1_results.get('buys_drift',     {}).get('p_value', 1)
bd_m = h1_results.get('buys_drift',     {}).get('mean_car', 0)

if not np.isnan(bd_p):
    if bd_p < 0.05 and bd_m > 0 and (np.isnan(bi_p) or bi_p > 0.10):
        print('RESULT: UNDERREACTION CONFIRMED — CAR[-1,+1] ≈ 0, CAR[+2,+20] > 0 (p < 0.05)')
        print('        Market initially ignores disclosures, then gradually incorporates information.')
    elif bd_p < 0.05 and bd_m > 0:
        print('RESULT: DRIFT SIGNIFICANT — but market also reacts at disclosure (both windows active)')
    elif bd_p > 0.10:
        print('RESULT: NO EXPLOITABLE SIGNAL — CAR[+2,+20] not statistically significant')
    else:
        print('RESULT: NEGATIVE DRIFT — unexpected direction; inspect data quality')
else:
    print('RESULT: Insufficient data for H1 determination.')

# ─── Visualization ───
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('H1 Event Study: CAR Around Disclosure Date (Event = ReportDate)',
             fontsize=14, fontweight='bold')

for ax, (direction, label, color) in zip(
    axes, [(1, 'Purchases', 'steelblue'), (-1, 'Sales', 'coral')]
):
    subset = df_cars[df_cars['Direction'] == direction]
    means  = [subset['CAR_immediate'].mean(), subset['CAR_drift'].mean()]
    sems   = [stats.sem(subset['CAR_immediate'].dropna()),
              stats.sem(subset['CAR_drift'].dropna())]
    x_labels = ['CAR[-1,+1]\n(Immediate)', 'CAR[+2,+20]\n(Drift)']

    bars = ax.bar(x_labels, means, yerr=[1.96 * s for s in sems],
                  color=color, alpha=0.7, capsize=10, edgecolor='white', linewidth=1.5)
    ax.axhline(0, color='black', linewidth=0.8)
    ax.set_title(f'{label} (N={len(subset):,})')
    ax.set_ylabel('Cumulative Abnormal Return')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    # Annotate bars
    for bar, val in zip(bars, means):
        if not np.isnan(val):
            ax.text(bar.get_x() + bar.get_width()/2., val + 0.001 * np.sign(val),
                    f'{val:.2%}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# H1 ROBUSTNESS CHECKS
# ═══════════════════════════════════════════════════════════════════════════

def run_robustness(cars_df: pd.DataFrame) -> dict:
    """Robustness: winsorized, lag filter, subperiod splits."""
    p1, p99 = cars_df['CAR_drift'].quantile([0.01, 0.99])
    cars_wins = cars_df.copy()
    cars_wins['CAR_drift'] = cars_wins['CAR_drift'].clip(p1, p99)

    # Lag data — merge DisclosureLag if not present
    if 'DisclosureLag' not in cars_df.columns:
        lag_data = df_enriched[['ReportDate', 'Ticker', 'DisclosureLag']].drop_duplicates()
        cars_with_lag = cars_df.merge(lag_data, on=['ReportDate', 'Ticker'], how='left')
    else:
        cars_with_lag = cars_df.copy()

    rd = pd.to_datetime(cars_df['ReportDate'])
    midpoint = rd.median()

    subsets = [
        ('Full Sample',        cars_df),
        ('Winsorized 1/99',    cars_wins),
        ('Lag ≤ 45 days',      cars_with_lag[cars_with_lag.get('DisclosureLag', pd.Series(dtype=float)).le(45)]
                               if 'DisclosureLag' in cars_with_lag.columns else cars_df),
        (f'Pre-{midpoint.year}',  cars_df[rd < midpoint]),
        (f'Post-{midpoint.year}', cars_df[rd >= midpoint]),
    ]

    results = {}
    for label, subset in subsets:
        buys = subset[subset['Direction'] == 1]['CAR_drift'].dropna()
        if len(buys) < 10:
            results[label] = {'n': len(buys), 'mean': np.nan, 't': np.nan, 'p': np.nan}
            continue
        t, p = stats.ttest_1samp(buys, 0)
        results[label] = {'n': len(buys), 'mean': buys.mean(), 't': t, 'p': p}
    return results


robustness = run_robustness(df_cars)

print('\n' + '=' * 70)
print('H1 ROBUSTNESS — CAR[+2,+20] for Purchases')
print('=' * 70)
print(f'{"Specification":<25} {"N":>8} {"Mean CAR":>10} {"t-stat":>8} {"p-value":>12}')
print('-' * 70)

for label, res in robustness.items():
    if np.isnan(res.get('mean', np.nan)):
        print(f'{label:<25} {res["n"]:>8,}  {"N/A":>10}')
        continue
    stars = '***' if res['p'] < 0.01 else '**' if res['p'] < 0.05 else '*' if res['p'] < 0.10 else ''
    print(f'{label:<25} {res["n"]:>8,} {res["mean"]*100:>9.2f}% {res["t"]:>8.3f} {res["p"]:>8.4f}{stars}')

---
## Phase 3: H2 — Committee Information Advantage

**Hypothesis:** Legislators trading in sectors relevant to their committee assignments generate higher post-disclosure CARs than other trades.

**Two required comparisons:**
- **Comparison A (Within-member):** Same legislator, committee-relevant vs. non-relevant trades
- **Comparison B (Cross-member):** Same sector, committee members vs. non-members

**H2 outcome determines committee weighting in signal engine.**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 11: H2 — Committee Information Advantage
# ═══════════════════════════════════════════════════════════════════════════

# Join committee relevance flags to CAR dataset
enrich_cols = ['ReportDate', 'Ticker', 'Is_Committee_Relevant', 'project_sector', 'Name']
df_cars_enriched = df_cars.merge(
    df_enriched[enrich_cols].drop_duplicates(subset=['ReportDate', 'Ticker']),
    on=['ReportDate', 'Ticker'], how='left'
)


def run_h2_cross_member(df: pd.DataFrame) -> dict:
    """
    Comparison B: For each sector, committee members vs. non-members.
    Test: difference in mean CAR_drift.
    """
    results = {}
    for sector in SECTOR_CONFIG:
        sdf = df[df['project_sector'] == sector]
        cmte     = sdf[sdf['Is_Committee_Relevant'] == True]['CAR_drift'].dropna()
        non_cmte = sdf[sdf['Is_Committee_Relevant'] == False]['CAR_drift'].dropna()

        if len(cmte) < 5 or len(non_cmte) < 5:
            results[sector] = {
                'n_committee': len(cmte), 'n_non': len(non_cmte),
                'mean_committee': np.nan, 'mean_non': np.nan,
                'diff': np.nan, 't': np.nan, 'p': np.nan,
            }
            continue

        t, p = stats.ttest_ind(cmte, non_cmte)
        results[sector] = {
            'n_committee':   len(cmte),
            'n_non':         len(non_cmte),
            'mean_committee':cmte.mean(),
            'mean_non':      non_cmte.mean(),
            'diff':          cmte.mean() - non_cmte.mean(),
            't':             t,
            'p':             p,
        }
    return results


def run_h2_within_member(df: pd.DataFrame) -> dict:
    """
    Comparison A: Within-member — legislators who trade both relevant and non-relevant.
    Compares mean CAR_drift for relevant vs. non-relevant trades per member.
    """
    name_col = 'Name_x' if 'Name_x' in df.columns else 'Name'
    members_with_both = (
        df.groupby(name_col)['Is_Committee_Relevant']
          .nunique()
          .pipe(lambda s: s[s > 1].index)
    )
    df_both = df[df[name_col].isin(members_with_both)]

    relevant     = df_both[df_both['Is_Committee_Relevant'] == True]['CAR_drift'].dropna()
    non_relevant = df_both[df_both['Is_Committee_Relevant'] == False]['CAR_drift'].dropna()

    if len(relevant) < 5 or len(non_relevant) < 5:
        return {'n_relevant': len(relevant), 'n_non': len(non_relevant),
                'diff': np.nan, 't': np.nan, 'p': np.nan}

    t, p = stats.ttest_ind(relevant, non_relevant)
    return {
        'n_relevant':   len(relevant),
        'n_non':        len(non_relevant),
        'n_legislators':len(members_with_both),
        'mean_relevant':relevant.mean(),
        'mean_non':     non_relevant.mean(),
        'diff':         relevant.mean() - non_relevant.mean(),
        't':            t,
        'p':            p,
    }


def run_h2_regression(df: pd.DataFrame):
    if sm is None:
        print('statsmodels unavailable; skipping H2 regression.')
        return None
    """
    OLS: CAR_drift = alpha + beta*Is_Committee_Relevant + controls
    Cluster SE by legislator.
    """
    name_col = 'Name_x' if 'Name_x' in df.columns else 'Name'
    df_reg = df.dropna(subset=['CAR_drift', 'Is_Committee_Relevant']).copy()
    df_reg['comm_int']  = df_reg['Is_Committee_Relevant'].astype(int)
    df_reg['buy']       = (df_reg['Direction'] == 1).astype(int)
    df_reg['log_amount']= np.log1p(df_reg.get('TotalAmount', pd.Series(1, index=df_reg.index)))

    if len(df_reg) < 20:
        print('Insufficient data for H2 regression.')
        return None

    try:
        X = sm.add_constant(df_reg[['comm_int', 'buy', 'log_amount']])
        y = df_reg['CAR_drift']
        groups = df_reg[name_col] if name_col in df_reg.columns else df_reg.index
        model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': groups})
        return model
    except Exception as e:
        print(f'H2 regression failed: {e}')
        return None


# ─── Run H2 ───
h2_within = run_h2_within_member(df_cars_enriched)
h2_cross  = run_h2_cross_member(df_cars_enriched)
h2_reg    = run_h2_regression(df_cars_enriched)

# Within-member
print('\n' + '=' * 70)
print('H2: WITHIN-MEMBER COMPARISON')
print('=' * 70)
if not np.isnan(h2_within.get('diff', np.nan)):
    print(f"Legislators trading in both relevant and non-relevant sectors: {h2_within.get('n_legislators', 'N/A')}")
    print(f"Mean CAR_drift (Committee-Relevant): {h2_within['mean_relevant']*100:.3f}%")
    print(f"Mean CAR_drift (Non-Relevant):       {h2_within['mean_non']*100:.3f}%")
    print(f"Difference:                          {h2_within['diff']*100:.3f}%")
    print(f"t-stat: {h2_within['t']:.3f} | p-value: {h2_within['p']:.4f}")
else:
    print('Insufficient within-member comparisons (requires legislators trading in both categories).')

# Cross-member
print('\n' + '=' * 80)
print('H2: CROSS-MEMBER COMPARISON BY SECTOR')
print('=' * 80)
print(f'{"Sector":<30} {"N(Cmte)":>8} {"N(Non)":>8} {"Δ CAR":>10} {"t":>8} {"p":>10}')
print('-' * 80)
for sector, res in h2_cross.items():
    if np.isnan(res.get('diff', np.nan)):
        print(f'{sector:<30} {res["n_committee"]:>8} {res["n_non"]:>8}  {"N/A":>10}')
        continue
    stars = '***' if res['p'] < 0.01 else '**' if res['p'] < 0.05 else '*' if res['p'] < 0.10 else ''
    print(f'{sector:<30} {res["n_committee"]:>8} {res["n_non"]:>8} '
          f'{res["diff"]*100:>9.2f}%{stars} {res["t"]:>8.3f} {res["p"]:>10.4f}')

# Regression
if h2_reg is not None:
    print('\n' + '=' * 60)
    print('H2 REGRESSION RESULTS (Clustered SE by Legislator)')
    print('=' * 60)
    coef  = h2_reg.params.get('comm_int', np.nan)
    pval  = h2_reg.pvalues.get('comm_int', np.nan)
    print(h2_reg.summary2().tables[1].round(4))
    print('\nINTERPRETATION:')
    if not np.isnan(pval):
        if pval < 0.05 and coef > 0:
            print('H2 CONFIRMED: Committee-relevant trades significantly outperform (p < 0.05)')
            print('DECISION: Apply committee filter in signal engine (already enforced in Module 12)')
        else:
            print('H2 NOT CONFIRMED: No significant committee information advantage')
            print('DECISION: Committee filter retained for risk management; equal weighting applied')

---
## Phase 4: Signal Engine

**Refactored from prior code — key fixes:**
1. **Bug fix:** Committee filter actually applied (`Is_Committee_Relevant == True` only)
2. **Bug fix:** `AmountMidpoint` used instead of raw `Amount` for volume calculations
3. **Enhancement:** `current_date` parameter enables zero-lookahead historical backtesting
4. **Enhancement:** `conviction_score` returned as continuous signal in `[-1, 1]`
5. **Fix:** `conviction_thresh` parameter not hardcoded — read from caller

**Signal logic:** For each sector, aggregate committee-relevant trades in `lookback_days` window ending at `current_date` (using `ReportDate`). Score = `net_volume / total_volume`. Gates: diversity (min members, min tickers) + volume (min transactions). Score ≥ threshold → BUY; ≤ −threshold → SELL; else FLAT.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 12: Signal Generator
# ═══════════════════════════════════════════════════════════════════════════

def generate_signals(
    df_enriched: pd.DataFrame,
    config: dict = None,
    lookback_days: int = LOOKBACK_DAYS,
    conviction_thresh: float = CONVICTION_THRESHOLD,
    current_date=None,
    committee_filter: bool = True,
) -> tuple:
    """
    Generate sector-level BUY/SELL/FLAT signals.

    CRITICAL: Uses ReportDate only — NO LOOKAHEAD from TransactionDate (ASSUMPTION A5).
    BUG FIX: committee_filter=True restricts to Is_Committee_Relevant trades only.
    BUG FIX: Uses AmountMidpoint (not raw Amount) for volume weighting (ASSUMPTION A1).
    ENHANCEMENT: conviction_score is continuous in [-1,1] for portfolio construction.
    ENHANCEMENT: current_date enables zero-lookahead historical simulation.

    Returns:
        (sector_summary_df, detailed_trades_df)
    """
    if config is None:
        config = SECTOR_CONFIG

    if current_date is None:
        current_date = df_enriched['ReportDate'].max()
    current_date   = pd.Timestamp(current_date)
    lookback_start = current_date - pd.Timedelta(days=lookback_days)

    # CRITICAL: Filter by ReportDate only — window ends at current_date
    window_df = df_enriched[
        (df_enriched['ReportDate'] >= lookback_start) &
        (df_enriched['ReportDate'] <= current_date)
    ].copy()

    # BUG FIX: Committee filter
    if committee_filter:
        window_df = window_df[window_df['Is_Committee_Relevant'] == True].copy()

    sector_results = []
    detailed_frames = []

    for sector, cfg in config.items():
        sdf = window_df[window_df['project_sector'] == sector].copy()

        if len(sdf) == 0:
            sector_results.append({
                'Sector':             sector,
                'target_etf':         cfg['target_etf'],
                'directive':          f'INSUFFICIENT DATA: No trades in {lookback_days}-day window',
                'conviction_score':   0.0,
                'total_transactions': 0,
                'net_volume':         0.0,
                'total_volume':       0.0,
                'n_members':          0,
                'n_tickers':          0,
                'diversity_pass':     False,
                'volume_pass':        False,
            })
            continue

        # BUG FIX: Volume calculated with AmountMidpoint
        sdf['signed_volume'] = sdf['Direction'] * sdf['AmountMidpoint']
        net_volume   = sdf['signed_volume'].sum()
        total_volume = sdf['AmountMidpoint'].sum()
        n_members    = sdf['Name'].nunique()
        n_tickers    = sdf['Ticker'].nunique()
        n_txn        = len(sdf)

        diversity_pass = (n_members >= cfg['tau_member']) and (n_tickers >= cfg['tau_ticker'])
        volume_pass    = n_txn >= cfg['min_transactions']

        # Conviction score: bounded [-1, 1]
        conviction_score = (net_volume / total_volume) if total_volume > 0 else 0.0
        conviction_score = float(np.clip(conviction_score, -1.0, 1.0))

        # Directive string
        if not volume_pass:
            directive = f'INSUFFICIENT DATA: {n_txn} transactions < {cfg["min_transactions"]} required'
        elif not diversity_pass:
            directive = f'FLAT: Diversity gate failed ({n_members} members, {n_tickers} tickers)'
        elif conviction_score >= conviction_thresh:
            directive = f'BUY LONG {cfg["target_etf"]}'
        elif conviction_score <= -conviction_thresh:
            directive = f'SELL SHORT {cfg["target_etf"]}'
        else:
            bias = 'bullish' if conviction_score > 0 else 'bearish'
            directive = f'FLAT: Conviction {conviction_score:.2f} below threshold ({bias})'

        sector_results.append({
            'Sector':             sector,
            'target_etf':         cfg['target_etf'],
            'directive':          directive,
            'conviction_score':   round(conviction_score, 4),
            'total_transactions': n_txn,
            'net_volume':         net_volume,
            'total_volume':       total_volume,
            'n_members':          n_members,
            'n_tickers':          n_tickers,
            'diversity_pass':     diversity_pass,
            'volume_pass':        volume_pass,
        })
        detailed_frames.append(sdf)

    sector_summary = pd.DataFrame(sector_results)
    detailed_df    = pd.concat(detailed_frames, ignore_index=True) if detailed_frames else pd.DataFrame()
    return sector_summary, detailed_df


# ─── Current Signal Dashboard ───
current_signals, current_detail = generate_signals(
    df_enriched,
    lookback_days=LOOKBACK_DAYS,
    conviction_thresh=CONVICTION_THRESHOLD,
    committee_filter=True,
)

print('\n' + '=' * 90)
print(f'SIGNAL DASHBOARD — {df_enriched["ReportDate"].max().date()}')
print(f'Lookback: {LOOKBACK_DAYS} days | Committee filter: ON | Threshold: {CONVICTION_THRESHOLD}')
print('=' * 90)
disp_cols = ['Sector', 'target_etf', 'directive', 'conviction_score',
             'total_transactions', 'n_members', 'n_tickers']
print(current_signals[disp_cols].to_string(index=False))

print('\n--- ACTIONABLE DIRECTIVES ---')
actionable = current_signals[current_signals['directive'].str.startswith(('BUY', 'SELL'))]
if actionable.empty:
    print('No sectors currently exceed conviction threshold.')
else:
    for _, row in actionable.iterrows():
        print(f'  {row["Sector"]}: {row["directive"]} (score={row["conviction_score"]:.3f})')

In [ ]:
# ─── Historical Signal Heatmap ───
print('Generating historical signal heatmap (month-end signals)...')
monthly_dates = pd.date_range(SAMPLE_START, df_enriched['ReportDate'].max(), freq='ME')

hist_signals = {}
for date in tqdm(monthly_dates, desc='Historical signals'):
    sigs, _ = generate_signals(
        df_enriched,
        lookback_days=BACKTEST_LOOKBACK_DAYS,
        conviction_thresh=CONVICTION_THRESHOLD,
        current_date=date,
        committee_filter=True,
    )
    for _, row in sigs.iterrows():
        hist_signals[(date, row['Sector'])] = row['conviction_score']

signal_matrix = (
    pd.DataFrame(
        [(d, s, v) for (d, s), v in hist_signals.items()],
        columns=['date', 'sector', 'conviction']
    )
    .pivot(index='date', columns='sector', values='conviction')
    .fillna(0)
)

fig, ax = plt.subplots(figsize=(16, 5))
sns.heatmap(
    signal_matrix.T,
    cmap='RdYlGn', center=0, vmin=-1, vmax=1,
    ax=ax,
    xticklabels=[
        d.strftime('%Y-%m') if i % 6 == 0 else ''
        for i, d in enumerate(signal_matrix.index)
    ],
    yticklabels=True,
    cbar_kws={'label': 'Conviction Score'},
    linewidths=0.3,
)
ax.set_title(
    'Historical Conviction Signal Heatmap by Sector\n'
    f'(Committee-relevant trades only | {BACKTEST_LOOKBACK_DAYS}-day lookback)',
    fontsize=13, fontweight='bold'
)
ax.set_xlabel('Month')
ax.set_ylabel('Sector')
plt.tight_layout()
plt.show()

---
## Phase 5: Backtest

**Primary Strategy — Variant A (Absolute Threshold):**
- Monthly rebalancing at month-end
- `generate_signals()` with 90-day lookback, committee filter ON
- Equal-weight sectors where conviction ≥ 0.80 → LONG
- Equal-weight sectors where conviction ≤ −0.80 → SHORT
- Zero-signal months → all cash (return = 0)
- Transaction cost: 10 bps one-way

**ASSUMPTION A8:** Threshold-based variant is primary. The hypothesis is absolute: a sector with a strong enough signal is tradeable independently. Tercile ranking fails with sparse signals (the common case with 7 sectors and strict gating).

**ASSUMPTION A9:** Strategy will be net-long most months — congressional buying clusters in bull markets. This is a signal property, not a bug. Factor regression alpha is the primary evaluation metric.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULES 13-14: Backtest — Portfolio Construction & Performance
# ═══════════════════════════════════════════════════════════════════════════

def run_backtest(
    df_enriched: pd.DataFrame,
    etf_returns: pd.DataFrame,
    config: dict = None,
    start_date: str = SAMPLE_START,
    end_date: str = SAMPLE_END,
    lookback_days: int = BACKTEST_LOOKBACK_DAYS,
    conviction_thresh: float = CONVICTION_THRESHOLD,
    rebalance_freq: str = 'ME',
    tc_bps: int = TRANSACTION_COST_BPS,
    variant: str = 'threshold',
) -> pd.DataFrame:
    """
    Full historical backtest with monthly rebalancing.

    CRITICAL: Signal at each rebalance_date uses only data UP TO rebalance_date.
              ETF returns for the holding period are applied AFTER the rebalance_date.
              No future information used in signal generation.

    variant: 'threshold' (primary) or 'tercile' (robustness)
    """
    if config is None:
        config = SECTOR_CONFIG

    rebalance_dates = pd.date_range(start_date, end_date, freq=rebalance_freq)
    etf_pivot = (
        etf_returns
        .pivot(index='date', columns='etf_ticker', values='ret')
        .pipe(lambda df: df.set_index(pd.to_datetime(df.index)))
        .sort_index()
    )

    records       = []
    prev_positions = {}

    for i, rebalance_date in enumerate(rebalance_dates[:-1]):
        next_date = rebalance_dates[i + 1]

        # CRITICAL: Signal uses only data up to rebalance_date
        signals, _ = generate_signals(
            df_enriched, config=config,
            lookback_days=lookback_days,
            conviction_thresh=conviction_thresh,
            current_date=rebalance_date,
            committee_filter=True,
        )

        # Determine positions
        if variant == 'threshold':
            long_etfs  = signals[
                signals['directive'].str.startswith('BUY') &
                signals['diversity_pass'] & signals['volume_pass']
            ]['target_etf'].tolist()
            short_etfs = signals[
                signals['directive'].str.startswith('SELL') &
                signals['diversity_pass'] & signals['volume_pass']
            ]['target_etf'].tolist()
        elif variant == 'tercile':
            valid = signals[signals['diversity_pass'] & signals['volume_pass']]
            if len(valid) < 3:
                long_etfs, short_etfs = [], []
            else:
                n_each    = len(valid) // 3
                sorted_v  = valid.sort_values('conviction_score', ascending=False)
                long_etfs  = sorted_v.head(n_each)['target_etf'].tolist()
                short_etfs = sorted_v.tail(n_each)['target_etf'].tolist()
        else:
            raise ValueError(f'Unknown variant: {variant}')

        # Equal-weight positions
        new_positions = {}
        if long_etfs:
            w = 1.0 / len(long_etfs)
            for etf in long_etfs:
                new_positions[etf] = new_positions.get(etf, 0) + w
        if short_etfs:
            w = -1.0 / len(short_etfs)
            for etf in short_etfs:
                new_positions[etf] = new_positions.get(etf, 0) + w

        # Turnover
        all_etfs = set(prev_positions) | set(new_positions)
        turnover = sum(
            abs(new_positions.get(e, 0) - prev_positions.get(e, 0))
            for e in all_etfs
        ) / 2

        # Portfolio return: compound daily returns over holding period
        period_rets = etf_pivot[
            (etf_pivot.index > rebalance_date) & (etf_pivot.index <= next_date)
        ]

        if period_rets.empty or not new_positions:
            port_ret_gross = 0.0
        else:
            daily_port = pd.Series(0.0, index=period_rets.index)
            for etf, weight in new_positions.items():
                if etf in period_rets.columns:
                    daily_port += weight * period_rets[etf].fillna(0)
            port_ret_gross = (1 + daily_port).prod() - 1

        tc             = turnover * tc_bps / 10_000
        port_ret_net   = port_ret_gross - tc

        records.append({
            'date':             next_date,
            'rebalance_date':   rebalance_date,
            'portfolio_ret_gross': port_ret_gross,
            'portfolio_ret_net':   port_ret_net,
            'long_etfs':        long_etfs,
            'short_etfs':       short_etfs,
            'n_long':           len(long_etfs),
            'n_short':          len(short_etfs),
            'n_valid_signals':  int((signals['diversity_pass'] & signals['volume_pass']).sum()),
            'turnover':         turnover,
        })
        prev_positions = new_positions

    return pd.DataFrame(records)


# ─── Run Primary Backtest ───
print('Running backtest (Variant A — Threshold)...')
backtest_results = run_backtest(
    df_enriched, etf_returns,
    start_date=SAMPLE_START, end_date=SAMPLE_END,
    lookback_days=BACKTEST_LOOKBACK_DAYS,
    conviction_thresh=CONVICTION_THRESHOLD,
    variant='threshold',
)
print(f'Complete: {len(backtest_results)} monthly periods')
print(f'Months with positions: {(backtest_results["n_long"] + backtest_results["n_short"] > 0).sum()}')
backtest_results.head()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# Performance Analytics & Plots
# ═══════════════════════════════════════════════════════════════════════════

def compute_performance(returns: pd.Series, rf_annual: float = 0.04) -> dict:
    """Annualized performance metrics for monthly return series."""
    periods = 12
    rf_monthly = rf_annual / periods

    ann_ret = (1 + returns).prod() ** (periods / len(returns)) - 1
    ann_vol = returns.std() * np.sqrt(periods)
    excess  = returns - rf_monthly
    sharpe  = (excess.mean() / returns.std()) * np.sqrt(periods) if returns.std() > 0 else np.nan

    cum = (1 + returns).cumprod()
    rolling_max = cum.cummax()
    dd = (cum - rolling_max) / rolling_max
    max_dd = dd.min()
    end_dd = dd.idxmin()
    start_dd = cum[:end_dd].idxmax() if not cum[:end_dd].empty else end_dd

    invested = (backtest_results['n_long'] + backtest_results['n_short'] > 0).mean()

    return {
        'annualized_return':       ann_ret,
        'annualized_vol':          ann_vol,
        'sharpe_ratio':            sharpe,
        'max_drawdown':            max_dd,
        'max_drawdown_start':      start_dd,
        'max_drawdown_end':        end_dd,
        'calmar_ratio':            ann_ret / abs(max_dd) if max_dd != 0 else np.nan,
        'hit_rate':                (returns > 0).mean(),
        'best_month':              returns.max(),
        'worst_month':             returns.min(),
        'n_months':                len(returns),
        'pct_months_invested':     invested,
        'avg_monthly_turnover':    backtest_results['turnover'].mean(),
    }


strategy_returns = backtest_results.set_index('date')['portfolio_ret_net']
perf = compute_performance(strategy_returns)

# SPY benchmark (monthly, compounded from daily)
spy_monthly = (
    etf_returns[etf_returns['etf_ticker'] == 'SPY']
    .set_index('date')['ret']
    .resample('ME')
    .apply(lambda x: (1 + x).prod() - 1)
    .rename('SPY')
)

print('\n' + '=' * 60)
print('BACKTEST PERFORMANCE SUMMARY (Variant A — Threshold)')
print('=' * 60)
for k, v in perf.items():
    if isinstance(v, float) and not np.isnan(v):
        if 'return' in k or 'vol' in k or 'drawdown' in k or 'rate' in k or 'month' in k or 'turnover' in k:
            print(f'  {k:<30} {v:>10.2%}')
        else:
            print(f'  {k:<30} {v:>10.3f}')
    elif hasattr(v, 'date'):
        print(f'  {k:<30} {v.date()}')
    else:
        print(f'  {k:<30} {v}')
print('=' * 60)

# ─── Plots ───
fig, axes = plt.subplots(3, 1, figsize=(16, 14))
fig.suptitle('Backtest — Congressional Trading Signal Strategy',
             fontsize=14, fontweight='bold')

# 1. Cumulative returns
ax = axes[0]
cum_strat = (1 + strategy_returns).cumprod()
cum_spy   = (1 + spy_monthly.reindex(strategy_returns.index).fillna(0)).cumprod()
ax.plot(cum_strat.index, cum_strat.values, label='Congressional Signal Strategy',
        linewidth=2, color='steelblue')
ax.plot(cum_spy.index, cum_spy.values, label='SPY Benchmark',
        linewidth=1.5, color='coral', linestyle='--')
ax.set_title('Cumulative Returns (Net of 10 bps Transaction Costs)')
ax.set_ylabel('Growth of $1')
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'${y:.2f}'))

# 2. Drawdown
ax = axes[1]
cum_ret = (1 + strategy_returns).cumprod()
rolling_max = cum_ret.cummax()
dd = (cum_ret - rolling_max) / rolling_max
ax.fill_between(dd.index, dd.values, 0, color='coral', alpha=0.6, label='Drawdown')
ax.set_title('Strategy Drawdown')
ax.set_ylabel('Drawdown')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
ax.legend()

# 3. Monthly returns heatmap
ax = axes[2]
mr = strategy_returns.copy()
mr.index = pd.to_datetime(mr.index)
pivot = (
    mr.to_frame('ret')
      .assign(year=lambda d: d.index.year, month=lambda d: d.index.month)
      .pivot(index='year', columns='month', values='ret')
)
pivot.columns = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
sns.heatmap(pivot, cmap='RdYlGn', center=0, annot=True, fmt='.1%',
            ax=ax, cbar_kws={'label': 'Monthly Return'}, linewidths=0.5)
ax.set_title('Monthly Returns Heatmap')

plt.tight_layout()
plt.show()

# Composition chart
fig, ax = plt.subplots(figsize=(10, 4))
n_pos = backtest_results['n_long'] + backtest_results['n_short']
ax.hist(n_pos, bins=range(0, max(n_pos) + 2), edgecolor='white',
        color='steelblue', alpha=0.8)
ax.set_title('Distribution of Active Sector Positions per Month')
ax.set_xlabel('Number of ETF Positions')
ax.set_ylabel('Number of Months')
plt.tight_layout()
plt.show()

# ─── Robustness: Variant B (Tercile) ───
print('\nRunning robustness check (Variant B — Tercile Ranking)...')
bt_tercile = run_backtest(
    df_enriched, etf_returns,
    start_date=SAMPLE_START, end_date=SAMPLE_END,
    lookback_days=BACKTEST_LOOKBACK_DAYS, conviction_thresh=CONVICTION_THRESHOLD,
    variant='tercile',
)
perf_t = compute_performance(bt_tercile.set_index('date')['portfolio_ret_net'])

print(f'\n{"Metric":<30} {"Threshold (Primary)":>22} {"Tercile (Robustness)":>22}')
print('-' * 78)
for k in ['annualized_return', 'annualized_vol', 'sharpe_ratio', 'max_drawdown']:
    v_a = perf.get(k, np.nan)
    v_b = perf_t.get(k, np.nan)
    fmt = '.2%' if any(x in k for x in ['return', 'vol', 'drawdown']) else '.3f'
    print(f'{k:<30} {v_a:>22{fmt}} {v_b:>22{fmt}}')

---
## Phase 6: Factor Regression — Alpha Evaluation

**Model:**
$$R_{portfolio,t} - R_{f,t} = \alpha + \beta_1\text{MktRF} + \beta_2\text{SMB} + \beta_3\text{HML} + \beta_4\text{RMW} + \beta_5\text{CMA} + \beta_6\text{Mom} + \epsilon_t$$

**Interpretation guide:**
- `α > 0, p < 0.05` → Strategy generates genuine alpha beyond known risk factors
- `α ≈ 0` → Returns are explained by factor exposures
- High `MktRF` loading → Strategy is primarily a directional market bet (expected per ASSUMPTION A9)
- High `Mom` loading → Strategy is momentum in disguise

In [33]:
# ═══════════════════════════════════════════════════════════════════════════
# MODULE 15: Factor Regression — FF5 + Momentum
# ═══════════════════════════════════════════════════════════════════════════

def run_factor_regression(portfolio_returns: pd.Series,
                          factor_data: pd.DataFrame,
                          frequency: str = 'monthly'):
    if sm is None:
        print('statsmodels unavailable; skipping factor regression.')
        return None
    """
    OLS regression of excess portfolio returns on FF5 + Momentum.
    portfolio_returns must be EXCESS returns (net of RF).
    factor_data must contain: Mkt_RF, SMB, HML, RMW, CMA, Mom.
    Returns (model, reg_data) or None.
    """
    if factor_data.empty:
        print('Factor data not available.')
        return None

    f = factor_data.copy()
    # Normalize column names
    f.columns = [c.strip().replace('-', '_').replace(' ', '') for c in f.columns]
    mom_col = next((c for c in f.columns if c.lower() in ('mom', 'wml', 'umd')), None)
    if mom_col and mom_col != 'Mom':
        f = f.rename(columns={mom_col: 'Mom'})

    # Align to monthly if needed
    if frequency == 'monthly':
        f = f.resample('ME').last() if f.index.freq != 'ME' else f
        f.index = f.index + pd.offsets.MonthEnd(0)

    required = [c for c in ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom'] if c in f.columns]
    common   = portfolio_returns.index.intersection(f.index)

    if len(common) < 12:
        print(f'Insufficient aligned months: {len(common)}')
        return None

    port = portfolio_returns.loc[common]
    fac  = f.loc[common]

    # Excess returns (subtract RF)
    rf = fac['RF'] if 'RF' in fac.columns else pd.Series(0.04 / 12, index=common)
    excess = port - rf

    reg_data = fac[required].copy()
    reg_data['excess_ret'] = excess
    reg_data = reg_data.dropna()

    if len(reg_data) < 12:
        print('Insufficient data after alignment.')
        return None

    X   = sm.add_constant(reg_data[required])
    y   = reg_data['excess_ret']
    res = sm.OLS(y, X).fit(cov_type='HC3')
    return res, reg_data


def format_regression_table(model) -> pd.DataFrame:
    """Format OLS results as a clean coefficient table."""
    def stars(p):
        return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

    tbl = pd.DataFrame({
        'Coefficient': model.params,
        'Std Error':   model.bse,
        't-stat':      model.tvalues,
        'p-value':     model.pvalues,
        'CI Lower':    model.conf_int()[0],
        'CI Upper':    model.conf_int()[1],
    })
    tbl['Sig'] = tbl['p-value'].apply(stars)
    return tbl.round(4)


# ─── Run Regression ───
ff_for_reg = ff_factors.copy() if isinstance(ff_factors, pd.DataFrame) else pd.DataFrame()

if ff_for_reg.empty:
    ff_cache_path = EXTERNAL_DIR / 'ff5_mom_factors.csv'
    if ff_cache_path.exists():
        ff_for_reg = pd.read_csv(ff_cache_path, index_col='date', parse_dates=True)
        print(f'Loaded factor cache for regression: {len(ff_for_reg):,} rows')

if ff_for_reg.empty:
    ff_for_reg = fetch_ff5_mom(ESTIMATION_WINDOW_START, SAMPLE_END)

if not ff_for_reg.empty:
    reg_result = run_factor_regression(strategy_returns, ff_for_reg, frequency='monthly')
else:
    reg_result = None
    print('Factor data unavailable — skipping regression.')
    print('No FF5+Momentum data from cache or download. Check network access to Ken French data library.')

if reg_result is not None:
    reg_model, reg_data = reg_result

    print('\n' + '=' * 70)
    print('FAMA-FRENCH 5 FACTOR + MOMENTUM REGRESSION')
    print(f'Sample: {reg_data.index.min().date()} to {reg_data.index.max().date()}')
    print(f'N = {len(reg_data)} monthly observations | HC3 standard errors')
    print('=' * 70)
    coef_table = format_regression_table(reg_model)
    print(coef_table.to_string())
    print(f'\nR²          = {reg_model.rsquared:.4f}')
    print(f'Adjusted R² = {reg_model.rsquared_adj:.4f}')

    alpha     = reg_model.params.get('const', np.nan)
    alpha_t   = reg_model.tvalues.get('const', np.nan)
    alpha_p   = reg_model.pvalues.get('const', np.nan)
    alpha_ann = (1 + alpha) ** 12 - 1 if not np.isnan(alpha) else np.nan
    mkt_beta  = reg_model.params.get('Mkt_RF', np.nan)

    print('\n' + '=' * 70)
    print('ALPHA ASSESSMENT')
    print('=' * 70)
    print(f'Monthly alpha:     {alpha:.4f}  ({alpha*10000:.1f} bps/month)')
    print(f'Annualized alpha:  {alpha_ann:.2%}')
    print(f't-statistic:       {alpha_t:.3f}')
    print(f'p-value:           {alpha_p:.4f}')
    print(f'Market beta:       {mkt_beta:.3f}')

    print('\nINTERPRETATION:')
    if not np.isnan(alpha_p):
        if alpha_p < 0.05 and alpha > 0:
            print('RESULT: SIGNIFICANT POSITIVE ALPHA — strategy adds value beyond risk factors (p < 0.05)')
        elif alpha_p < 0.10 and alpha > 0:
            print('RESULT: MARGINAL POSITIVE ALPHA — borderline significant (p < 0.10)')
        elif alpha_p > 0.10 and alpha > 0:
            print('RESULT: POSITIVE BUT INSIGNIFICANT ALPHA — returns partially explained by factor tilts')
        elif alpha <= 0:
            print('RESULT: NON-POSITIVE ALPHA — strategy does not add value after factor adjustment')

    if not np.isnan(mkt_beta) and abs(mkt_beta) > 0.5:
        print(f'\nNOTE: High MktRF loading ({mkt_beta:.2f}) confirms directional market exposure')
        print('      (expected per ASSUMPTION A9 — congressional buying clusters in bull markets)')

pandas_datareader not installed — skipping factor download.
Factor data unavailable — skipping regression.
No FF5+Momentum data from cache or download. Check network access to Ken French data library.


In [32]:
# ─── Factor Loading Visualization ───
if reg_result is not None:
    reg_model, reg_data = reg_result
    coef_table = format_regression_table(reg_model)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Factor Regression Results — FF5 + Momentum', fontsize=14, fontweight='bold')

    # Factor loadings with 95% CI
    ax = axes[0]
    params = coef_table[coef_table.index != 'const'].copy()
    y_pos  = range(len(params))
    colors = ['steelblue' if c > 0 else 'coral' for c in params['Coefficient']]
    ax.barh(y_pos, params['Coefficient'],
            xerr=1.96 * params['Std Error'],
            color=colors, alpha=0.7, capsize=5, edgecolor='white')
    ax.axvline(0, color='black', linewidth=0.8)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(params.index)
    ax.set_title('Factor Loadings (±1.96 SE)')
    ax.set_xlabel('Coefficient')

    # Alpha highlighted separately
    alpha_val = coef_table.loc['const', 'Coefficient'] if 'const' in coef_table.index else 0
    alpha_se  = coef_table.loc['const', 'Std Error']   if 'const' in coef_table.index else 0
    ax.text(0.95, 0.05,
            f'α = {alpha_val:.4f}\n(ann. {(1+alpha_val)**12-1:.1%})',
            transform=ax.transAxes, ha='right', va='bottom',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

    # Fitted vs actual
    ax = axes[1]
    fitted = reg_model.fittedvalues
    actual = reg_data['excess_ret']
    ax.scatter(fitted, actual, alpha=0.5, color='steelblue', s=30)
    lims = [min(fitted.min(), actual.min()), max(fitted.max(), actual.max())]
    ax.plot(lims, lims, 'r--', linewidth=1, label='45° line')
    ax.set_title('Fitted vs Actual Excess Returns')
    ax.set_xlabel('Fitted (Factor Model)')
    ax.set_ylabel('Actual Portfolio Excess Return')
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    ax.legend()

    plt.tight_layout()
    plt.show()


### Assumptions Made
| ID | Assumption | Impact if Wrong |
|----|-----------|----------------|
| A1 | Amount field treated as midpoint of STOCK Act range | Conviction scores shift; threshold may need recalibration |
| A2 | Restrict to 7 sectors with high government intervention | May miss alpha in other sectors |
| A3 | GICS classification is current, not point-in-time | Some tickers misclassified in historical periods |
| A4 | Committee assignments at Congress session level, not exact date | Some trades tagged to wrong committee |
| A5 | Lookback window uses ReportDate only | Signal may be stale for trades with long disclosure lag |
| A6 | Dedup: composite key includes Amount | Trade counts slightly inflated for truly identical trades |
| A7 | Transaction cost: 10 bps one-way | If costs are higher, net returns decrease proportionally |
| A8 | Threshold-based portfolio (Variant A) is primary strategy | Strategy carries directional market exposure |
| A9 | Strategy net-long most months (congressional buying in bull markets) | High MktRF beta in factor regression |
| A10 | API column names normalized immediately: Filed→ReportDate, Traded→TransactionDate | Pipeline crashes if rename is missed |


### Unresolved Questions
1. **Committee roster quality:** Full H2 analysis requires `CONGRESS_API_KEY` from congress.gov
2. **Point-in-time GICS:** Currently using current sector classification (ASSUMPTION A3)
3. **Lookback sensitivity:** Primary analysis uses 90-day window per proposal; robustness checks needed
4. **Delisting survivorship:** Tickers delisted during the sample period may bias event study results

### Next Required Artifacts
- `logs/assumptions.md` — Formal log with all assumption IDs, dates, and rationale
- `logs/data_transformations.md` — Row counts at each pipeline stage
- Backtest return time series saved for external validation
- Factor regression output saved to file